  ## Проект: Обнаружение и сокрытие лиц (блюр/пикселизация)

---

## 1. Введение

С увеличением объёма видеоматериалов, публикуемых в интернете и снимаемых с камер наблюдения, защита личной информации становится критически важной.  

Данный проект направлен на создание автоматизированной системы для **детекции лиц и их анонимизации** с использованием методов **блюра и пикселизации**.  
Система должна работать как на отдельных изображениях, так и на видео, обеспечивая устойчивость к различным условиям: движениям, плохому освещению и многолюдным сценам.

---

## 2. Проблема исследования

Существующие методы сокрытия лиц имеют несколько ограничений:

- Низкая точность детекции при сложных условиях  
- Сложность обработки лиц при частичной окклюзии (маски, очки, волосы)  
- Ограниченные возможности настройки уровня анонимизации (степень блюра или размер пикселей)  
- Ограничения существующих решений. Например:

  - Некоторые API дорогие или имеют ограниченные возможности настройки.  
  - Другие решения не обеспечивают достаточную точность или стабильность в сложных сценариях, таких как:
    - частичная окклюзия лиц (маски, очки, волосы)  
    - шумное видео  
    - низкое разрешение кадров  
  - Многие готовые сервисы закрытые (не open-source), что затрудняет дообучение на собственных данных.

Проект направлен на устранение этих проблем через создание гибкой, обучаемой и развёртываемой системы.

---

## 3. Цель исследования

**Основная цель:** разработать систему, которая автоматически обнаруживает лица и сокрывает их с помощью блюра или пикселизации, обеспечивая точность, стабильность и готовность к развертыванию.

**Подцели:**

1. Подготовка и организация датасетов для обучения и тестирования 
2. Обучение модели на обьединеных датасетах
3. Тестирование и оценка качества системы с использованием метрик
4. Реализация алгоритмов детекции и анонимизации (блюр)  
5. Проведение дообучения моделей для повышения точности  



---

### Шаг 1. Подготовка и организация датасета
- Скачивание данных ( WIDER FACE)  
- Организация структуры папок для тренировки и валидации (train/val)  
- Преобразование аннотаций в формат YOLO  
- Аугментация данных:
  - Повороты и зеркальное отражение  
  - Случайная обрезка  
  - Эффекты движения и размытия  
  - Частичная окклюзия и низкое освещение  

In [11]:

# ===============================
# 1. Установка необходимых библиотек
# ===============================

# Обновление pip
%pip install --upgrade pip # Обновление pip до последней версии

# Для YOLO11
%pip install ultralytics # Установка библиотеки ultralytics для работы с моделями YOLO

# Для обработки изображений и видео
%pip install opencv-python opencv-python-headless # Установка OpenCV для компьютерного зрения

# Для научных вычислений
%pip install matplotlib # Установка Matplotlib для визуализации данных






Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
# ===============================
# 2. Определение путей и параметров
# ===============================
SRC_ROOT = "/root/project/DarkFace_Train_2021" # Корневая папка с исходными данными
IMAGES_SRC = f"{SRC_ROOT}/image" # Папка с изображениями
LABELS_SRC = f"{SRC_ROOT}/label" # Папка с аннотациями

OUT_ROOT = "/root/DarkFace_YOLO" # Корневая папка для выходных данных в формате YOLO

TRAIN_RATIO = 0.8 # Доля данных для обучения
VAL_RATIO = 0.2   # Доля данных для валидации
CLASS_ID = 0  # ID класса для всех объектов (лиц) в датасете


In [13]:
# Загрузка датасета WIDER FACE через kagglehub
import kagglehub

print("⏳ Загружаем датасет WIDER FACE...") # сообщение о начале загрузки
path =  kagglehub.dataset_download("lylmsc/wider-face-for-yolo-training") # скачивание датасета
print("📁 Датасет скачан в:", path)

/opt/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


⏳ Загружаем датасет WIDER FACE...


100%|██████████| 2.45G/2.45G [01:28<00:00, 29.6MB/s]

Extracting files...


📁 Датасет скачан в: /root/.cache/kagglehub/datasets/lylmsc/wider-face-for-yolo-training/versions/1


In [14]:
# ===============================
# 3. Проверка структуры датасета
# ===============================
import os

base = "/root/.cache/kagglehub/datasets/lylmsc/wider-face-for-yolo-training/versions/1" # базовый путь к датасету

print("📁 images/:", os.listdir(f"{base}/images"))
print("📁 labels/:", os.listdir(f"{base}/labels"))


📁 images/: ['wider_0.jpg', 'wider_1.jpg', 'wider_10.jpg', 'wider_100.jpg', 'wider_1000.jpg', 'wider_10000.jpg', 'wider_10001.jpg', 'wider_10002.jpg', 'wider_10003.jpg', 'wider_10004.jpg', 'wider_10005.jpg', 'wider_10006.jpg', 'wider_10007.jpg', 'wider_10008.jpg', 'wider_10009.jpg', 'wider_1001.jpg', 'wider_10010.jpg', 'wider_10011.jpg', 'wider_10012.jpg', 'wider_10013.jpg', 'wider_10014.jpg', 'wider_10015.jpg', 'wider_10016.jpg', 'wider_10017.jpg', 'wider_10018.jpg', 'wider_10019.jpg', 'wider_1002.jpg', 'wider_10020.jpg', 'wider_10021.jpg', 'wider_10022.jpg', 'wider_10023.jpg', 'wider_10024.jpg', 'wider_10025.jpg', 'wider_10026.jpg', 'wider_10027.jpg', 'wider_10028.jpg', 'wider_10029.jpg', 'wider_1003.jpg', 'wider_10030.jpg', 'wider_10031.jpg', 'wider_10032.jpg', 'wider_10033.jpg', 'wider_10034.jpg', 'wider_10035.jpg', 'wider_10036.jpg', 'wider_10037.jpg', 'wider_10038.jpg', 'wider_10039.jpg', 'wider_1004.jpg', 'wider_10040.jpg', 'wider_10041.jpg', 'wider_10042.jpg', 'wider_10043.jpg',

In [15]:
# ===============================
#  Просмотр структуры датасета
# ===============================
import os

ROOT = "/root/project/DarkFace_Train_2021" # Путь к корневой папке датасета

print("📂 ROOT:", os.listdir(ROOT))    

print("\n📂 images:")
print(os.listdir(f"{ROOT}/image")[:20])

print("\n📂 labels:")
print(os.listdir(f"{ROOT}/label")[:20])

# посмотрим расширения
img_exts = set(os.path.splitext(f)[1] for f in os.listdir(f"{ROOT}/image")) # расширения изображений
lbl_exts = set(os.path.splitext(f)[1] for f in os.listdir(f"{ROOT}/label")) # расширения аннотаций

print("\n🖼 image extensions:", img_exts)
print("🏷 label extensions:", lbl_exts)


📂 ROOT: ['label', 'image', 'yolo_ready']

📂 images:
['1.png', '10.png', '100.png', '1000.png', '1001.png', '1002.png', '1003.png', '1004.png', '1005.png', '1006.png', '1007.png', '1008.png', '1009.png', '101.png', '1010.png', '1011.png', '1012.png', '1013.png', '1014.png', '1015.png']

📂 labels:
['1.txt', '10.txt', '100.txt', '1000.txt', '1001.txt', '1005.txt', '1004.txt', '1003.txt', '1002.txt', '1006.txt', '1007.txt', '1008.txt', '101.txt', '1009.txt', '1010.txt', '1011.txt', '1012.txt', '1014.txt', '1013.txt', '1016.txt']

🖼 image extensions: {'.png'}
🏷 label extensions: {'.txt'}


In [16]:
# ===============================
# 4. Подготовка датасета в формате YOLO
# ===============================
import os
import shutil
import random

SRC_ROOT = "/root/project/DarkFace_Train_2021"
OUT_ROOT = "/root/project/DarkFace_Train_2021/yolo_ready"

IMG_SRC = os.path.join(SRC_ROOT, "image")
LBL_SRC = os.path.join(SRC_ROOT, "label")

IMG_EXTS = (".jpg", ".png", ".jpeg") # допустимые расширения изображений

# Create dirs
for p in [
    f"{OUT_ROOT}/images/train", # папка для тренировочных изображений
    f"{OUT_ROOT}/images/val", # папка для валидационных изображений
    f"{OUT_ROOT}/labels/train", # папка для тренировочных аннотаций
    f"{OUT_ROOT}/labels/val", # папка для валидационных аннотаций
]:
    os.makedirs(p, exist_ok=True)

# Поиск пар изображений и аннотаций
pairs = [] # список для хранения имен файлов с изображениями, для которых есть аннотации
for f in os.listdir(IMG_SRC):   # перебор всех файлов в папке с изображениями
    if f.lower().endswith(IMG_EXTS): # проверка расширения файла
        lbl = f.rsplit(".", 1)[0] + ".txt" # формирование имени файла аннотации
        if os.path.exists(os.path.join(LBL_SRC, lbl)): # проверка существования файла аннотации
            pairs.append(f) # добавление имени файла изображения в список, если аннотация существует

print(f"🔎 Found image+label pairs: {len(pairs)}")

random.shuffle(pairs) # перемешивание списка пар для случайного распределения
split = int(len(pairs) * 0.9) # определение индекса для разделения на тренировочную и валидационную выборки
train, val = pairs[:split], pairs[split:] # разделение на тренировочную и валидационную выборки

def copy(files, split): # функция для копирования файлов в соответствующие папки
    for f in files: # перебор всех файлов в переданном списке
        shutil.copy( 
            os.path.join(IMG_SRC, f), # копирование изображения
            os.path.join(OUT_ROOT, "images", split, f) # в соответствующую папку (train/val)
        )
        shutil.copy(
            os.path.join(LBL_SRC, f.rsplit(".", 1)[0] + ".txt"), # копирование аннотации
            os.path.join(OUT_ROOT, "labels", split, f.rsplit(".", 1)[0] + ".txt") # в соответствующую папку (train/val)
        )

copy(train, "train")
copy(val, "val")

print(f"📊 train={len(train)} | val={len(val)}")


🔎 Found image+label pairs: 6000
📊 train=5400 | val=600


In [17]:
# ===============================
#  Преобразование датасета в формат YOLO12
# ===============================
import os
import shutil
import random
import yaml

base = "/root/.cache/kagglehub/datasets/lylmsc/wider-face-for-yolo-training/versions/1" # базовый путь к датасету

src_images = os.path.join(base, "images") # папка с исходными изображениями
src_labels = os.path.join(base, "labels") # папка с исходными аннотациями

yolo_base = os.path.join(base, "yolo_ready")  # новая структура
img_train = os.path.join(yolo_base, "images/train") # папка для тренировочных изображений
img_val = os.path.join(yolo_base, "images/val") # папка для валидационных изображений
lbl_train = os.path.join(yolo_base, "labels/train") # папка для тренировочных аннотаций
lbl_val = os.path.join(yolo_base, "labels/val") # папка для валидационных аннотаций

# Создаём папки
for p in [img_train, img_val, lbl_train, lbl_val]: # создание необходимых папок
    os.makedirs(p, exist_ok=True)

# Список всех изображений
all_images = [f for f in os.listdir(src_images) if f.endswith(".jpg")] # получение списка всех изображений с расширением .jpg
random.shuffle(all_images) # перемешивание списка изображений

split_index = int(len(all_images) * 0.8) # определение индекса для разделения на тренировочную и валидационную выборки
train_files = all_images[:split_index] # файлы для тренировки
val_files = all_images[split_index:] # файлы для валидации

# Перенос train
for f in train_files:
    shutil.copy(os.path.join(src_images, f), os.path.join(img_train, f)) # копирование изображения в папку train
    lbl = f.replace(".jpg", ".txt") # формирование имени файла аннотации
    shutil.copy(os.path.join(src_labels, lbl), os.path.join(lbl_train, lbl)) # копирование аннотации в папку train

# Перенос val
for f in val_files:
    shutil.copy(os.path.join(src_images, f), os.path.join(img_val, f)) # копирование изображения в папку val
    lbl = f.replace(".jpg", ".txt") # формирование имени файла аннотации
    shutil.copy(os.path.join(src_labels, lbl), os.path.join(lbl_val, lbl)) # копирование аннотации в папку val

print("✓ Train images:", len(os.listdir(img_train)))
print("✓ Train labels:", len(os.listdir(lbl_train)))
print("✓ Val images:", len(os.listdir(img_val)))
print("✓ Val labels:", len(os.listdir(lbl_val)))

# Создаём data.yaml
data_yaml = os.path.join(yolo_base, "data.yaml") # путь к файлу data.yaml
 
# Конфигурация для data.yaml
config = { 
    "train": img_train,
    "val": img_val,
    "nc": 1,
    "names": ["face"]
}

with open(data_yaml, "w") as f: # запись конфигурации в файл data.yaml
    yaml.dump(config, f) 

print("\n✔ data.yaml создан:", data_yaml)
print("\n🎉 Датасет готов к обучению YOLO12!")


✓ Train images: 10304
✓ Train labels: 10304
✓ Val images: 2576
✓ Val labels: 2576

✔ data.yaml создан: /root/.cache/kagglehub/datasets/lylmsc/wider-face-for-yolo-training/versions/1/yolo_ready/data.yaml

🎉 Датасет готов к обучению YOLO12!


In [77]:
# ===============================
# DarkFace (x1 y1 x2 y2) → YOLO
# ===============================
import os
import cv2

ROOT = "/root/project/DarkFace_Train_2021/yolo_ready"
IMG_ROOT = f"{ROOT}/images"
LBL_ROOT = f"{ROOT}/labels"

def convert_one(img_path, lbl_path): # функция для конвертации одной аннотации
    img = cv2.imread(img_path) # чтение изображения
    if img is None: # проверка успешности чтения изображения
        return False 

    h, w, _ = img.shape # получение размеров изображения
    new_lines = []

    with open(lbl_path) as f: # чтение аннотации
        lines = f.readlines() 

    # skip first line (number of faces)
    for line in lines[1:]: # перебор всех строк, кроме первой
        parts = line.strip().split() # разбиение строки на части
        if len(parts) != 4: # проверка корректности количества частей
            continue

        x1, y1, x2, y2 = map(float, parts) # преобразование координат в числа с плавающей точкой

        bw = x2 - x1 # ширина бокса
        bh = y2 - y1 # высота бокса
        xc = x1 + bw / 2 # центр по x
        yc = y1 + bh / 2 # центр по y

        # normalize
        xc /= w # нормализация центра по x
        yc /= h # нормализация центра по y
        bw /= w # нормализация ширины бокса
        bh /= h # нормализация высоты бокса

        if 0 < xc < 1 and 0 < yc < 1 and 0 < bw < 1 and 0 < bh < 1: # проверка корректности нормализованных значений
            new_lines.append(f"0 {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}") # добавление строки в формате YOLO

    if not new_lines: # проверка наличия корректных аннотаций
        return False

    with open(lbl_path, "w") as f: # запись конвертированных аннотаций обратно в файл
        f.write("\n".join(new_lines))

    return True


ok, bad = 0, 0 # счетчики успешных и неуспешных конвертаций

for split in ["train", "val"]: # перебор тренировочной и валидационной выборок
    img_dir = f"{IMG_ROOT}/{split}" 
    lbl_dir = f"{LBL_ROOT}/{split}"

    for lbl in os.listdir(lbl_dir): # перебор всех файлов аннотаций
        img = lbl.replace(".txt", ".png")
        img_path = os.path.join(img_dir, img)
        lbl_path = os.path.join(lbl_dir, lbl)

        if not os.path.exists(img_path): # проверка существования изображения
            bad += 1
            continue

        if convert_one(img_path, lbl_path): # попытка конвертации аннотации
            ok += 1
        else:
            bad += 1

print(f"✅ Converted labels: {ok}")
print(f"❌ Dropped images: {bad}")


✅ Converted labels: 6000
❌ Dropped images: 0


In [ ]:
# ===============================
# 5. Объединение DarkFace и WIDER FACE в один датасет
# ===============================
import os       # Работа с файловой системой: папки, пути
import shutil   # Копирование файлов
import random   # Перемешивание списка файлов

# Папка, куда будем объединять датасеты
OUT = "/root/project/faces_combined"

# Создаём структуру папок для объединённого датасета
os.makedirs(f"{OUT}/images/train", exist_ok=True)  # train images
os.makedirs(f"{OUT}/images/val", exist_ok=True)    # val images
os.makedirs(f"{OUT}/labels/train", exist_ok=True)  # train labels
os.makedirs(f"{OUT}/labels/val", exist_ok=True)    # val labels

# Словарь с исходными датасетами и их префиксами для имён файлов
datasets = {
    "df": "/root/project/DarkFace_Train_2021/yolo_ready",  # DarkFace
    "wf": "/root/.cache/kagglehub/datasets/lylmsc/wider-face-for-yolo-training/versions/1/yolo_ready"  # WIDER FACE
}

TRAIN_RATIO = 0.8  # Доля данных для train (80%), остальное для val

all_pairs = []  # Сюда будем собирать все пары (image + label)

# Проходим по каждому датасету
for prefix, root in datasets.items():
    # Проходим по train и val папкам исходного датасета
    for split in ["train", "val"]:
        img_dir = f"{root}/images/{split}"  # путь к картинкам
        lbl_dir = f"{root}/labels/{split}"  # путь к лейблам

        # Проходим по всем файлам изображений
        for img in os.listdir(img_dir):
            lbl = img.rsplit(".", 1)[0] + ".txt"  # формируем имя соответствующего .txt
            # Проверяем, что лейбл существует
            if os.path.exists(f"{lbl_dir}/{lbl}"):
                # Добавляем в список: префикс, путь к картинке, путь к лейблу, имя картинки
                all_pairs.append((prefix, img_dir, lbl_dir, img))

# Перемешиваем список, чтобы train/val не шли блоками
random.shuffle(all_pairs)

# Вычисляем индекс для разбиения на train и val
split = int(len(all_pairs) * TRAIN_RATIO)

# Функция копирования изображений и лейблов в новую объединённую структуру
def copy(pairs, split):
    for prefix, img_dir, lbl_dir, img in pairs:
        new_name = f"{prefix}_{img}"  # добавляем префикс к имени файла
        # Копируем изображение в новую папку
        shutil.copy(f"{img_dir}/{img}", f"{OUT}/images/{split}/{new_name}")
        # Копируем соответствующий лейбл
        shutil.copy(
            f"{lbl_dir}/{img.rsplit('.',1)[0]}.txt",
            f"{OUT}/labels/{split}/{new_name.rsplit('.',1)[0]}.txt"
        )

# Копируем первые split элементов в train
copy(all_pairs[:split], "train")
# Копируем оставшиеся элементы в val
copy(all_pairs[split:], "val")

# Вывод статистики
print(f"📊 Total images: {len(all_pairs)}")      # общее количество файлов
print(f"📊 Train: {split}")                     # сколько в train
print(f"📊 Val: {len(all_pairs)-split}")       # сколько в val


In [ ]:
import yaml
import os

# Папка объединённого датасета
ROOT = "/root/project/faces_combined"

# Словарь для data.yaml
data = {
    "path": ROOT,           # путь к корню датасета
    "train": "images/train", # папка с train изображениями
    "val": "images/val",     # папка с val изображениями
    "names": {
        0: "face"           # класс: лицо
    }
}

# Создаём файл data.yaml
yaml_path = os.path.join(ROOT, "data.yaml")
with open(yaml_path, "w") as f:
    yaml.dump(data, f, sort_keys=False)

print(f"✅ data_combined.yaml создан успешно: {yaml_path}")


### Шаг 2. Обучение модели на обьединеных датасетах
- Создание `data.yaml` и запуск тренировки выбранной модели на подготовленном датасете  
- Настройка гиперпараметров: epochs, batch size, learning rate, img size  
- Валидация модели на тестовом наборе данных

In [ ]:
# ===============================
# 1. Создадим data.yaml файл для обучения
# ===============================
import yaml
import os

ROOT = "/root/project/DarkFace_Train_2021/yolo_ready"
# Создание структуры данных для data.yaml
data = { 
    "path": ROOT,
    "train": "images/train",
    "val": "images/val",
    "names": {
        0: "face"
    }
}

with open(os.path.join(ROOT, "data.yaml"), "w") as f: # запись данных в data.yaml
    yaml.dump(data, f, sort_keys=False) # сохранение без сортировки ключей

print("✅ data.yaml создан успешно")


✅ data.yaml создан успешно


In [67]:
import os

root = "/root/project/DarkFace_Train_2021"
print("ROOT:", os.listdir(root))

print("\nimage:", os.listdir(os.path.join(root, "image")) if os.path.exists(os.path.join(root, "image")) else "NO image/")
print("images:", os.listdir(os.path.join(root, "images")) if os.path.exists(os.path.join(root, "images")) else "NO images/")


ROOT: ['label', 'image', 'yolo_ready']

image: ['1.png', '10.png', '100.png', '1000.png', '1001.png', '1002.png', '1003.png', '1004.png', '1005.png', '1006.png', '1007.png', '1008.png', '1009.png', '101.png', '1010.png', '1011.png', '1012.png', '1013.png', '1014.png', '1015.png', '1016.png', '1017.png', '1018.png', '1019.png', '102.png', '1020.png', '1021.png', '1022.png', '1023.png', '1024.png', '1025.png', '1026.png', '1027.png', '1028.png', '1029.png', '103.png', '1031.png', '1030.png', '1032.png', '1033.png', '1034.png', '1035.png', '1036.png', '1037.png', '1038.png', '1039.png', '104.png', '1040.png', '1041.png', '1042.png', '1043.png', '1044.png', '1045.png', '1046.png', '1047.png', '1048.png', '1049.png', '105.png', '1050.png', '1051.png', '1052.png', '1053.png', '1054.png', '1055.png', '1056.png', '1057.png', '1058.png', '1059.png', '106.png', '1060.png', '1061.png', '1062.png', '1063.png', '1064.png', '1065.png', '1066.png', '1067.png', '1068.png', '1069.png', '107.png', '1070

In [68]:
import os

root = "/root/project/DarkFace_Train_2021/yolo_ready"

for p in ["images", "labels"]:
    print(p, "exists:", os.path.exists(os.path.join(root, p)))
    if os.path.exists(os.path.join(root, p)):
        print(" ", os.listdir(os.path.join(root, p))[:5])


images exists: True
  ['train', 'val']
labels exists: True
  ['train', 'val']


In [75]:
# Список файлов в labels/val и пример содержимого одного из файлов
import os

labels_val_dir = "/root/project/DarkFace_Train_2021/yolo_ready/labels/val"

if os.path.exists(labels_val_dir):
    label_files = os.listdir(labels_val_dir)[:5]  # Show first 5 files
    print(f"📂 Found {len(os.listdir(labels_val_dir))} label files in val/")
    print(f"📄 Sample files: {label_files}\n")
    
    # Read first label file as example
    if label_files:
        sample_file = os.path.join(labels_val_dir, label_files[0])
        with open(sample_file) as f:
            print(f"Content of {label_files[0]}:")
            print(f.read())
else:
    print(f"❌ Directory does not exist: {labels_val_dir}")


📂 Found 600 label files in val/
📄 Sample files: ['4205.txt', '4297.txt', '3386.txt', '3222.txt', '4819.txt']

Content of 4205.txt:
12
274 461 284 471
355 452 367 468
446 459 457 468
474 456 485 466
573 465 580 471
510 459 514 465
524 461 527 465
517 461 521 465
538 461 541 465
299 469 314 479
550 463 553 468
556 462 561 465



In [76]:
path = "/root/project/DarkFace_Train_2021/yolo_ready/labels/val/4205.txt"
with open(path) as f:
    print(f.read())


12
274 461 284 471
355 452 367 468
446 459 457 468
474 456 485 466
573 465 580 471
510 459 514 465
524 461 527 465
517 461 521 465
538 461 541 465
299 469 314 479
550 463 553 468
556 462 561 465



In [ ]:
# ===============================
# 2. Просмотр базовых метрик на валидации DarkFace (до обучения)
# ===============================

from ultralytics import YOLO
import os

# Загружаем предобученную face-модель
model = YOLO("yolov12s-face.pt")  

data_yaml_path = "/root/project/DarkFace_Train_2021/yolo_ready/data.yaml"

# Check if paths exist
print(f"📂 Checking paths:")
print(f"data.yaml exists: {os.path.exists(data_yaml_path)}")
print(f"images/train exists: {os.path.exists('/root/project/DarkFace_Train_2021/yolo_ready/images/train')}")
print(f"images/val exists: {os.path.exists('/root/project/DarkFace_Train_2021/yolo_ready/images/val')}")
print(f"labels/train exists: {os.path.exists('/root/project/DarkFace_Train_2021/yolo_ready/labels/train')}")
print(f"labels/val exists: {os.path.exists('/root/project/DarkFace_Train_2021/yolo_ready/labels/val')}")

# Валидация
metrics = model.val(
    data=data_yaml_path,
    imgsz=640,  # размер изображения
    batch=16,
    device=0
)

print("\n📊 BASELINE METRICS (before training):")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall:    {metrics.box.mr:.4f}")
print(f"mAP50:     {metrics.box.map50:.4f}")
print(f"mAP50-95:  {metrics.box.map:.4f}")


📂 Checking paths:
data.yaml exists: True
images/train exists: True
images/val exists: True
labels/train exists: True
labels/val exists: True
Ultralytics 8.3.233 🚀 Python-3.12.11 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32109MiB)
YOLOv12s summary (fused): 159 layers, 9,231,267 parameters, 0 gradients, 21.2 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 8516.9±549.0 MB/s, size: 795.4 KB)
val: Scanning /root/project/DarkFace_Train_2021/yolo_ready/labels/val.cache... 600 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 600/600 1.6Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 38/38 18.8it/s 2.0s0.1s
                   all        600       4951      0.558      0.227      0.248     0.0786
Speed: 0.4ms preprocess, 1.3ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /root/project/runs/detect/val18

📊 BASELINE METRICS (before training):
Precision: 0.5581
Recall:    0.2268
mAP50

In [ ]:
# Просмотр базовых метрик на валидации Wider Face (до обучения)
from ultralytics import YOLO
import os
base = "/root/.cache/kagglehub/datasets/lylmsc/wider-face-for-yolo-training/versions/1" # базовый путь к датасету
data_yaml_path = "/root/.cache/kagglehub/datasets/lylmsc/wider-face-for-yolo-training/versions/1/yolo_ready/data.yaml" # путь к файлу data.yaml
model = YOLO("yolov12s-face.pt")  # загрузка предобученной модели YOLO12s-face
# Валидация
metrics = model.val(
    data=data_yaml_path,
    imgsz=640,  # размер изображения
    batch=16,
    device=0
)
print("\n📊 BASELINE METRICS (before training):")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall:    {metrics.box.mr:.4f}")
print(f"mAP50:     {metrics.box.map50:.4f}")
print(f"mAP50-95:  {metrics.box.map:.4f}")


Ultralytics 8.3.233 🚀 Python-3.12.11 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32109MiB)
YOLOv12s summary (fused): 159 layers, 9,231,267 parameters, 0 gradients, 21.2 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4740.5±1063.7 MB/s, size: 278.3 KB)
val: Scanning /root/.cache/kagglehub/datasets/lylmsc/wider-face-for-yolo-training/versions/1/yolo_ready/labels/val... 9487 images, 2 backgrounds, 1 corrupt: 100% ━━━━━━━━━━━━ 9487/9487 3.6Kit/s 2.6s<0.0s
val: /root/.cache/kagglehub/datasets/lylmsc/wider-face-for-yolo-training/versions/1/yolo_ready/images/val/wider_10969.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0253906]
val: /root/.cache/kagglehub/datasets/lylmsc/wider-face-for-yolo-training/versions/1/yolo_ready/images/val/wider_12381.jpg: 1 duplicate labels removed
val: /root/.cache/kagglehub/datasets/lylmsc/wider-face-for-yolo-training/versions/1/yolo_ready/images/val/wider_7207.jpg: 1 duplicate labels removed
val: New cache crea

In [ ]:
# Просмотр базовых метрик на валидации на обьединеных датасетах (до обучения)
from ultralytics import YOLO
import os
base = "/root/project/faces_combined"
data_yaml_path = "/root/project/faces_combined/data.yaml"
model = YOLO("yolov12s-face.pt")  # загрузка предобученной модели YOLO12s-face
# Валидация
metrics = model.val(
    data=data_yaml_path,
    imgsz=640,  # размер изображения
    batch=16,
    device=0
)
print("\n📊 BASELINE METRICS (before training):")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall:    {metrics.box.mr:.4f}")
print(f"mAP50:     {metrics.box.map50:.4f}")
print(f"mAP50-95:  {metrics.box.map:.4f}")


Ultralytics 8.3.233 🚀 Python-3.12.11 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32109MiB)
YOLOv12s summary (fused): 159 layers, 9,231,267 parameters, 0 gradients, 21.2 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 6850.4±2643.6 MB/s, size: 386.9 KB)
val: Scanning /root/project/faces_combined/labels/val.cache... 5290 images, 0 backgrounds, 1 corrupt: 100% ━━━━━━━━━━━━ 5290/5290 13.5Mit/s 0.0s
val: /root/project/faces_combined/images/val/wf_wider_10969.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0253906]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 331/331 25.0it/s 13.2s0.1s
                   all       5289      63881      0.844      0.562      0.637      0.363
Speed: 0.3ms preprocess, 0.9ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /root/project/runs/detect/val26

📊 BASELINE METRICS (before training):
Precision: 0.8444
Recall:    0.5619
mAP50

In [81]:
import os

LBL_ROOT = "/root/project/DarkFace_Train_2021/yolo_ready/labels"

total_faces = 0 # счетчик общего количества лиц
total_images = 0 # счетчик общего количества изображений

for split in ["train", "val"]: # перебор тренировочной и валидационной выборок
    lbl_dir = os.path.join(LBL_ROOT, split) # путь к папке с аннотациями

    for f in os.listdir(lbl_dir): # перебор всех файлов аннотаций
        if not f.endswith(".txt"): # проверка расширения файла
            continue

        with open(os.path.join(lbl_dir, f)) as file: # чтение аннотации
            lines = [l for l in file.readlines() if l.strip()] # получение непустых строк

        total_faces += len(lines) # добавление количества лиц в текущем файле
        total_images += 1 # увеличение счетчика изображений

avg_faces = total_faces / total_images # вычисление среднего количества лиц на изображение, путем деления общего количества лиц на общее количество изображений

print(f"📸 Images: {total_images}")
print(f"🙂 Faces:  {total_faces}")
print(f"📊 Avg faces per image: {avg_faces:.2f}")


📸 Images: 6000
🙂 Faces:  50396
📊 Avg faces per image: 8.40


In [ ]:
# ===============================
# 3. Обучение модели Yolo12 на обьединеном датасете
# ===============================
from ultralytics import YOLO
import torch
import os

# путь к твоему data.yaml
data_yaml = "/root/project/faces_combined/data.yaml"

# автоматически выбираем GPU
device = 0 if torch.cuda.is_available() else "cpu" # выбор устройства: GPU (0) или CPU
print("🔥 Использую устройство:", device)

# загружаем модель YOLO12s-face
model = YOLO("yolov12s-face.pt") # загрузка предобученной модели YOLO12s-face

# обучение
model.train( 
    data=data_yaml, # путь к data.yaml
    epochs=100, # количество эпох
    imgsz=640, # размер изображения
    batch=32, # размер батча
    device=device, # использование выбранного устройства
    workers=4, #количество рабочих потоков для загрузки данных
    lr0=0.01,        # начальный learning rate
    optimizer="SGD", # оптимизатор
    cache=True    # кэширование данных для ускорения обучения
)

print("🎉 Обучение завершено!")


🔥 Использую устройство: 0
New https://pypi.org/project/ultralytics/8.3.247 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.233 🚀 Python-3.12.11 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32109MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/root/project/faces_combined/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov12s-face.pt, momentum=0.937, mosaic=1.0, multi_

### Шаг 3. Метрики
- Метрики: mAP, IoU, Recall, Precision, FPS  



In [21]:
import os
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Force CPU mode to avoid CUDA compatibility issues
os.environ['CUDA_VISIBLE_DEVICES'] = ''

# ================================================
# CONFIG
# ================================================
MODEL_PATH = "/root/project/runs/detect/train/weights/best.pt"
yolo_base = "/root/project/faces_combined"
DATA_YAML = os.path.join(yolo_base, "data.yaml")
# Use existing validation images instead of non-existent test_images
TEST_IMAGES = "/root/project/faces_combined/images/val"
TEST_VIDEO = None                                     # "test.mp4" если нужно
REPORT_DIR = "model_report"

os.makedirs(REPORT_DIR, exist_ok=True)


# ================================================
def evaluate_map():
    model = YOLO(MODEL_PATH)
    metrics = model.val(data=DATA_YAML, imgsz=640, device='cpu')

    results = {
        "mAP_50": metrics.box.map50,
        "mAP_50_95": metrics.box.map,
        "Precision": metrics.box.mp,
        "Recall": metrics.box.mr
    }

    # сохраняем
    with open(f"{REPORT_DIR}/metrics.txt", "w") as f:
        for k, v in results.items():
            f.write(f"{k}: {v:.4f}\n")

    print("📊 Метрики сохранены в metrics.txt")
    return results


# ================================================
# 2. ФУНКЦИЯ: FPS тест
# ================================================
def evaluate_fps(video_path=None):
    model = YOLO(MODEL_PATH)

    if video_path is None:
        print("⚠ FPS тест пропущен: нет видео.")
        return None

    cap = cv2.VideoCapture(video_path)

    frame_count = 0
    start = time.time()

    while True:
        ret, frame = cap.read()
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        model(frame, device='cpu')
        frame_count += 1

    with open(f"{REPORT_DIR}/fps.txt", "w") as f:
        f.write(f"FPS: {fps:.2f}")

    print(f"⚡ FPS = {fps:.2f}")
    return fps


# ================================================
# 3. ВИЗУАЛЬНОЕ СРАВНЕНИЕ ДО/ПОСЛЕ
# ================================================
def generate_visual_comparison():
    model = YOLO(MODEL_PATH)

    out_dir = f"{REPORT_DIR}/comparison"
    os.makedirs(out_dir, exist_ok=True)

    for name in os.listdir(TEST_IMAGES):
        if not name.lower().endswith((".jpg", ".png", ".jpeg")):
            continue

        path = os.path.join(TEST_IMAGES, name)
        img = cv2.imread(path)
        orig = img.copy()

        results = model(img)[0]

        for box in results.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)

        combined = np.hstack([orig, img])
        cv2.imwrite(os.path.join(out_dir, name), combined)

    print("🖼 Сравнение сохранено в папку comparison/")


# ================================================
# 4. СТАБИЛЬНОСТЬ БОКСА (дрожание)
# ================================================
def evaluate_stability(video_path=None):
    if video_path is None:
        print("⚠ Тест стабильности пропущен: нет видео")
        return

    model = YOLO(MODEL_PATH)

    cap = cv2.VideoCapture(video_path)

    prev_center = None
    jitters = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)[0]
        if len(results.boxes.xyxy) == 0:
            continue

        # берём самое большое лицо
        boxes = results.boxes.xyxy.cpu().numpy()
        box = sorted(
            boxes,
            key=lambda b: (b[2]-b[0])*(b[3]-b[1]),
            reverse=True
        )[0]

        x1, y1, x2, y2 = box
        center = ((x1+x2)/2, (y1+y2)/2)

        if prev_center is not None:
            dx = center[0] - prev_center[0]
            dy = center[1] - prev_center[1]
            jitters.append(np.sqrt(dx*dx + dy*dy))

        prev_center = center

    if jitters:
        jitter_score = np.mean(jitters)
    else:
        jitter_score = 0

    with open(f"{REPORT_DIR}/stability.txt", "w") as f:
        f.write(f"Среднее дрожание (px/frame): {jitter_score:.2f}")

    print(f"🧊 Стабильность бокса: {jitter_score:.2f}px")


# ================================================
# 5. Чтение логов обучения и построение графиков
# ================================================
def plot_training_curves():
    # YOLO автоматически сохраняет curves.png, мы просто копируем
    train_dir = "/root/runs/detect/train5"  # поменяй путь если нужно

    src = os.path.join(train_dir, "results.png")
    dst = os.path.join(REPORT_DIR, "training_results.png")

    if os.path.exists(src):
        import shutil
        shutil.copy(src, dst)
        print("📈 Графики обучения скопированы в отчёт")
    else:
        print("⚠ Не найден results.png – YOLO мог сохранить его в другом месте.")

# ================================================
# 6. Функция построения графика метрик
# ================================================

def plot_metrics(metrics):
    plt.figure(figsize=(8, 5))
    
    names = ["mAP50", "mAP50-95", "Precision", "Recall"]
    values = [
        metrics["mAP_50"],
        metrics["mAP_50_95"],
        metrics["Precision"],
        metrics["Recall"]
    ]

    plt.bar(names, values, color=["#4CAF50", "#2196F3", "#FFC107", "#FF5722"])
    plt.ylim(0, 1)
    plt.title("Model Evaluation Metrics")
    plt.ylabel("Score")
    plt.grid(axis="y", linestyle="--", alpha=0.5)

    out = os.path.join(REPORT_DIR, "metrics_barplot.png")
    plt.savefig(out)
    plt.show()

    print("📊 График метрик сохранён в:", out)




print("\n🎉 ГОТОВО! Полный отчёт в папке model_report/")
print("\n📊 Метрики:")
print(f" - Precision: {metrics['Precision']:.4f}")
print(f" - Recall: {metrics['Recall']:.4f}")
print(f" - mAP50: {metrics['mAP_50']:.4f}")
print(f" - mAPP50-95: {metrics['mAP_50_95']:.4f}")






🎉 ГОТОВО! Полный отчёт в папке model_report/

📊 Метрики:
 - Precision: 0.6689
 - Recall: 0.3071
 - mAP50: 0.3552
 - mAPP50-95: 0.1288


### Шаг 4. Детекция и блюр
- Обнаружение лиц на изображениях и кадрах видео  
- Применение Gaussian blur или пикселизации к областям лиц  


In [ ]:
# Детекция на примере изображения из валидационного набора
from ultralytics import YOLO
import cv2         # OpenCV для работы с изображениями
import os          # Работа с файловой системой
import numpy as np # Для работы с массивами и объединения изображений

# ===============================
# 1. Папки для изображений и коллажей
# ===============================
IMG_DIR = "/root/project/faces_combined/images/val"  # Папка с исходными изображениями
OUT_DIR = "/root/project/faces_combined/blur_collages"  # Папка для сохранения коллажей
os.makedirs(OUT_DIR, exist_ok=True)  # Создаём папку, если она ещё не существует

# ===============================
# 2. Настройки размытия
# ===============================
BLUR_KERNEL = (25, 25)  # Размер ядра для GaussianBlur, чем больше – тем сильнее размытие

# ===============================
# 3. Проходим по всем файлам в папке
# ===============================
for img_name in os.listdir(IMG_DIR):  # Перебираем все файлы в папке
    if not img_name.lower().endswith((".png", ".jpg", ".jpeg")):  # Игнорируем не изображения
        continue  # Пропускаем файл, если это не изображение

    # Полный путь к изображению
    img_path = os.path.join(IMG_DIR, img_name)
    # Загружаем изображение
    img = cv2.imread(img_path)
    if img is None:  # Если изображение не удалось загрузить
        continue  # Пропускаем этот файл

    # ===============================
    # 4. Создаём размытую версию изображения
    # ===============================
    blurred = cv2.GaussianBlur(img, BLUR_KERNEL, 0)  # Применяем GaussianBlur

    # ===============================
    # 5. Создаём овальную маску
    # ===============================
    mask = np.zeros_like(img, dtype=np.uint8)  # Черная маска того же размера, что изображение
    h, w = img.shape[:2]  # Высота и ширина изображения
    center = (w // 2, h // 2)  # Центр овала (по центру изображения)
    axes = (w // 3, h // 3)  # Полуоси овала (ширина/2, высота/2)
    # Рисуем белый овал на черной маске
    cv2.ellipse(mask, center, axes, angle=0, startAngle=0, endAngle=360, color=(255, 255, 255), thickness=-1)
    # thickness=-1 означает, что овал будет залит белым полностью

    # ===============================
    # 6. Объединяем оригинал и размытие через маску
    # ===============================
    mask_float = mask.astype(float) / 255.0  # Преобразуем маску в диапазон [0,1]
    # Формируем изображение: размытие там, где белый овал, оригинал там, где черная область
    masked_blur = (blurred * mask_float + img * (1 - mask_float)).astype(np.uint8)

    # ===============================
    # 7. Создаём коллаж "оригинал | размытие с маской"
    # ===============================
    collage = np.hstack((img, masked_blur))  # Горизонтальное объединение изображений

    # ===============================
    # 8. Сохраняем коллаж
    # ===============================
    out_path = os.path.join(OUT_DIR, img_name)  # Путь сохранения
    cv2.imwrite(out_path, collage)  # Сохраняем коллаж

# ===============================
# 9. Вывод сообщения о завершении
# ===============================
print(f"✅ Коллажи сохранены в {OUT_DIR}")


✅ Коллажи сохранены в /root/project/faces_combined/blur_collages


In [8]:
# Детекция на примере изображения из своих данных
from ultralytics import YOLO
import cv2         # OpenCV для работы с изображениями
import os          # Работа с файловой системой
import numpy as np # Для работы с массивами и объединения изображений

# ===============================
# 1. Папки для изображений и коллажей
# ===============================
IMG_DIR = "/root/project/photo"  # Папка с исходными изображениями
OUT_DIR = "/root/project/faces_combined/blur_my_collages"  # Папка для сохранения коллажей
os.makedirs(OUT_DIR, exist_ok=True)  # Создаём папку, если она ещё не существует

# ===============================
# 2. Настройки размытия
# ===============================
BLUR_KERNEL = (51, 51)  # Размер ядра для GaussianBlur, чем больше – тем сильнее размытие

# ===============================
# 3. Проходим по всем файлам в папке
# ===============================
for img_name in os.listdir(IMG_DIR):  # Перебираем все файлы в папке
    if not img_name.lower().endswith((".png", ".jpg", ".jpeg")):  # Игнорируем не изображения
        continue  # Пропускаем файл, если это не изображение

    # Полный путь к изображению
    img_path = os.path.join(IMG_DIR, img_name)
    # Загружаем изображение
    img = cv2.imread(img_path)
    if img is None:  # Если изображение не удалось загрузить
        continue  # Пропускаем этот файл

    # ===============================
    # 4. Создаём размытую версию изображения
    # ===============================
    blurred = cv2.GaussianBlur(img, BLUR_KERNEL, 0)  # Применяем GaussianBlur

    # ===============================
    # 5. Создаём овальную маску
    # ===============================
    mask = np.zeros_like(img, dtype=np.uint8)  # Черная маска того же размера, что изображение
    h, w = img.shape[:2]  # Высота и ширина изображения
    center = (w // 2, h // 2)  # Центр овала (по центру изображения)
    axes = (w // 3, h // 3)  # Полуоси овала (ширина/2, высота/2)
    # Рисуем белый овал на черной маске
    cv2.ellipse(mask, center, axes, angle=0, startAngle=0, endAngle=360, color=(255, 255, 255), thickness=-1)
    # thickness=-1 означает, что овал будет залит белым полностью

    # ===============================
    # 6. Объединяем оригинал и размытие через маску
    # ===============================
    mask_float = mask.astype(float) / 255.0  # Преобразуем маску в диапазон [0,1]
    # Формируем изображение: размытие там, где белый овал, оригинал там, где черная область
    masked_blur = (blurred * mask_float + img * (1 - mask_float)).astype(np.uint8)

    # ===============================
    # 7. Создаём коллаж "оригинал | размытие с маской"
    # ===============================
    collage = np.hstack((img, masked_blur))  # Горизонтальное объединение изображений

    # ===============================
    # 8. Сохраняем коллаж
    # ===============================
    out_path = os.path.join(OUT_DIR, img_name)  # Путь сохранения
    cv2.imwrite(out_path, collage)  # Сохраняем коллаж

# ===============================
# 9. Вывод сообщения о завершении
# ===============================
print(f"✅ Коллажи сохранены в {OUT_DIR}")


✅ Коллажи сохранены в /root/project/faces_combined/blur_my_collages


In [2]:
# Блюр лиц на изображениях с помощью YOLO-face
import cv2              # OpenCV — работа с изображениями
import os               # Работа с файлами и папками
import numpy as np      # Массивы и операции с изображениями
from ultralytics import YOLO  # YOLO — детекция лиц


# ===============================
# 1️⃣ Пути к папкам
# ===============================

IMG_DIR = "/root/project/faces_combined/images/val" # Папка с исходными изображениями
OUT_DIR = "/root/project/faces_combined/blur_collages1" # Папка, куда сохраняем коллаж
os.makedirs(OUT_DIR, exist_ok=True) # Создаём папку, если её нет


# ===============================
# 2️⃣ Загрузка YOLO-face модели
# ===============================

YOLO_WEIGHTS = "/root/project/runs/detect/train3/weights/best.pt" # Путь к обученной YOLO-face модели
model = YOLO(YOLO_WEIGHTS) # Загружаем модель в память


# ===============================
# 3️⃣ Параметры размытия
# ===============================

BLUR_KERNEL = (51, 51) # Размер ядра размытия (должен быть НЕЧЁТНЫМ)

CORNER_RADIUS = 0.25 # Радиус скругления углов
# 0.25 = 25% от меньшей стороны bbox


# ===============================
# 4️⃣ Функция создания скруглённой маски
# ===============================

def rounded_mask(h, w, radius_ratio):
    """
    Создаёт маску со скруглёнными углами
    h, w — размеры лица
    radius_ratio — процент скругления
    """

    # Создаём чёрную маску
    mask = np.zeros((h, w), dtype=np.uint8)

    # Радиус скругления в пикселях
    r = int(min(h, w) * radius_ratio)

    # Рисуем центральный прямоугольник
    cv2.rectangle(mask, (r, 0), (w - r, h), 255, -1) # горизонтальный
    cv2.rectangle(mask, (0, r), (w, h - r), 255, -1) # вертикальный

    # Рисуем 4 круга по углам
    cv2.circle(mask, (r, r), r, 255, -1) # верхний левый
    cv2.circle(mask, (w - r, r), r, 255, -1) # верхний правый
    cv2.circle(mask, (r, h - r), r, 255, -1) # нижний левый
    cv2.circle(mask, (w - r, h - r), r, 255, -1) # нижний правый

    return mask


# ===============================
# 5️⃣ Основной цикл по изображениям
# ===============================

for img_name in os.listdir(IMG_DIR):

    # Берём только изображения
    if not img_name.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    # Полный путь к файлу
    img_path = os.path.join(IMG_DIR, img_name)

    # Загружаем изображение
    img = cv2.imread(img_path)
    if img is None:
        continue

    # Размеры изображения
    h, w = img.shape[:2]

    # Копия изображения для результата
    result = img.copy()


    # ===============================
    # 6️⃣ Детекция лиц YOLO
    # ===============================

    preds = model(
        img,
        conf=0.4,       # минимальная уверенность
        iou=0.5,        # IoU threshold
        verbose=False
    )[0]

    # Если лиц нет — сохраняем оригинал
    if preds.boxes is None:
        continue


    # ===============================
    # 7️⃣ Проходим по каждому лицу
    # ===============================

    for box in preds.boxes.xyxy.cpu().numpy():

        # Координаты bbox
        x1, y1, x2, y2 = map(int, box)

        # Защита от выхода за границы
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(w, x2)
        y2 = min(h, y2)

        # Вырезаем лицо
        face = img[y1:y2, x1:x2]

        if face.size == 0:
            continue


        # ===============================
        # 8️⃣ Размываем лицо
        # ===============================

        face_blur = cv2.GaussianBlur(face, BLUR_KERNEL, 0)

        fh, fw = face.shape[:2]

        # Создаём скруглённую маску
        mask = rounded_mask(fh, fw, CORNER_RADIUS)

        # Приводим маску к 3 каналам
        mask_3c = cv2.merge([mask, mask, mask]) / 255.0

        # Комбинируем blur и оригинал через маску
        blended = (
            face_blur * mask_3c +
            face * (1 - mask_3c)
        ).astype(np.uint8)

        # Вставляем обратно в изображение
        result[y1:y2, x1:x2] = blended


    # ===============================
    # 9️⃣ Создаём коллаж
    # ===============================

    collage = np.hstack([
        img,        # оригинал
        result      # со скруглённым blur
    ])


    # ===============================
    # 🔟 Сохраняем
    # ===============================

    out_path = os.path.join(OUT_DIR, img_name)
    cv2.imwrite(out_path, collage)


# ===============================
# ✅ Готово
# ===============================

print(f"✅ Готово! Коллажи сохранены в папке:\n{OUT_DIR}")


✅ Готово! Коллажи сохранены в папке:
/root/project/faces_combined/blur_collages1


### Шаг 5. Сглаживание bounding box
- Применение скользящего среднего, экспоненциального сглаживания или фильтра Калмана  
- Снижение дрожания боксов и улучшение визуального качества на видео
- Сокрытие всех лиц кроме целевого(по эталонному изображению) c помощью face_recognition

In [ ]:
# Блюр лиц в видео с помощью YOLO + DeepSORT
import cv2
import numpy as np
import subprocess
import os
import yt_dlp
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort


# =====================================================
# 1. СКАЧИВАНИЕ YOUTUBE ВИДЕО
# =====================================================

def download_video(url, filename="input.mp4"): # Скачивает видео с YouTube по ссылке
    if os.path.exists(filename): # Проверяет, существует ли файл
        print(f"✓ Файл уже существует: {filename}") # Если файл существует, выводит сообщение и
        return filename # возвращает имя файла

    print("📥 Скачиваю видео с YouTube...")

    ydl_opts = { 
        "format": "best[ext=mp4][height<=720]", # Выбирает лучшее качество до 720p в формате mp4
        "outtmpl": filename, # Устанавливает имя выходного файла
        "noplaylist": True # Отключает скачивание плейлистов
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl: # Инициализирует yt_dlp с заданными опциями
            ydl.download([url]) # Скачивает видео по ссылке
        print(f"✔ Видео скачано: {filename}") # Выводит сообщение об успешном скачивании
    except Exception as e: # Обрабатывает исключения
        print("❌ Ошибка загрузки:", e) # Выводит сообщение об ошибке
        raise

    return filename


# =====================================================
# 2. СКРУГЛЕННЫЙ БЛЮР
# =====================================================

def blur_rounded(img, box, blur_k=61, radius=40): # Применяет скруглённый блюр к области изображения, заданной box
    h_img, w_img = img.shape[:2] # Получает размеры изображения
    x1, y1, x2, y2 = map(int, box) # Преобразует координаты box в целые числа
# Ограничивает координаты рамки размерами изображения
    x1 = max(0, min(x1, w_img - 1)) 
    y1 = max(0, min(y1, h_img - 1)) 
    x2 = max(0, min(x2, w_img - 1))
    y2 = max(0, min(y2, h_img - 1))

    if x2 <= x1 or y2 <= y1: # Проверяет корректность координат
        return img

    face = img[y1:y2, x1:x2] # Вырезает область лица из изображения
    if face.size == 0:
        return img

    blurred = cv2.GaussianBlur(face, (blur_k, blur_k), 0) # Применяет GaussianBlur к области лица

    h, w = face.shape[:2] # Получает размеры области лица
    mask = np.zeros((h, w), dtype=np.uint8) # Создаёт чёрную маску
# Создаёт маску с закруглёнными углами
    cv2.rectangle(mask, (radius, 0), (w-radius, h), 255, -1)
    cv2.rectangle(mask, (0, radius), (w, h-radius), 255, -1)
    cv2.circle(mask, (radius, radius), radius, 255, -1)
    cv2.circle(mask, (w-radius, radius), radius, 255, -1)
    cv2.circle(mask, (radius, h-radius), radius, 255, -1)
    cv2.circle(mask, (w-radius, h-radius), radius, 255, -1)

    mask3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR) / 255.0 # Преобразует маску в 3 канала и нормализует
    img[y1:y2, x1:x2] = (blurred * mask3 + face * (1 - mask3)).astype(np.uint8) # Комбинирует размытое лицо с оригиналом через маску

    return img


# =====================================================
# 3. АВТО-ПЕРЕКОДИРОВКА AV1 → H.264
# =====================================================

def ensure_h264(input_path): # Проверяет кодек видео и при необходимости перекодирует AV1 в H.264
    cmd = [
        "ffprobe", "-v", "error", # Использует ffprobe для получения информации о видео
        "-select_streams", "v:0", # Выбирает первый видеопоток
        "-show_entries", "stream=codec_name", # Запрашивает имя кодека
        "-of", "default=noprint_wrappers=1:nokey=1", # Форматирует вывод
        input_path
    ]

    try:
        codec = subprocess.check_output(cmd).decode().strip() # Выполняет команду и получает кодек
    except: # Обрабатывает ошибки выполнения команды
        print("⚠ Не удалось определить кодек, продолжаю...")
        return input_path

    print("🎥 Кодек обнаружен:", codec)

    if codec.lower() == "av1": # Если кодек AV1, выполняет перекодировку
        print("❗ AV1 → перекодировка в H.264...")
        out_path = input_path.replace(".mp4", "_h264.mp4") # Создаёт имя для перекодированного файла

        convert_cmd = [ # Команда ffmpeg для перекодировки
            "ffmpeg", "-y", # Перезаписывает выходной файл без запроса
            "-i", input_path, # Входной файл
            "-c:v", "libx264", # Использует кодек H.264
            "-preset", "fast", # Предустановка скорости кодирования
            "-c:a", "copy", # Копирует аудиопоток без изменений
            out_path # Выходной файл
        ]
        subprocess.run(convert_cmd) # Выполняет команду перекодировки
        print("✔ Готово:", out_path) # Выводит сообщение об успешной перекодировке
        return out_path

    return input_path


# =====================================================
# 4. YOLO + DEEPSORT + БЛЮР
# =====================================================

def process_video(input_path, output_path, model_path): # Обрабатывает видео, размывая лица с помощью YOLO и DeepSORT
    print("🚀 Запускаю обработку видео...")

    model = YOLO(model_path) # Загружает модель YOLO из указанного пути

    tracker = DeepSort( # Инициализирует трекер DeepSORT с заданными параметрами
        max_age=30, # максимальное количество кадров для хранения трека без обновления
        n_init=3, # количество последовательных подтверждений для инициализации трека
        nms_max_overlap=0.7, # максимальное перекрытие для NMS
        max_iou_distance=0.7, # максимальное IoU для сопоставления
        embedder="mobilenet", # используемый эмбеддер
        embedder_gpu=True, # использование GPU для эмбеддера
        half=True # использование половинной точности
    )

    cap = cv2.VideoCapture(input_path) # Открывает входное видео
    w, h = int(cap.get(3)), int(cap.get(4)) # Получает ширину и высоту видео
    fps = cap.get(5) # Получает частоту кадров видео

    writer = cv2.VideoWriter( # Инициализирует VideoWriter для сохранения выходного видео
        output_path, # путь к выходному файлу
        cv2.VideoWriter_fourcc(*"mp4v"), # кодек видео
        fps, # частота кадров
        (w, h) # размер кадра
    )

    while True:
        ret, frame = cap.read() # Читает кадр из видео
        if not ret: # Если кадры закончились, выходит из цикла
            break

        results = model(frame, conf=0.3)[0] # Выполняет детекцию лиц с помощью YOLO

        detections = [] # Список для хранения детекций
        for box, conf in zip(results.boxes.xyxy, results.boxes.conf): # Проходит по каждой детекции
            x1, y1, x2, y2 = box.cpu().numpy() # Получает координаты бокса
            w_box, h_box = x2 - x1, y2 - y1 # Вычисляет ширину и высоту бокса

            if w_box > 0 and h_box > 0: # Проверяет корректность размеров бокса
                detections.append([[x1, y1, w_box, h_box], float(conf.cpu().numpy())]) # Добавляет детекцию в список

        tracks = tracker.update_tracks(detections, frame=frame) # Обновляет треки с помощью DeepSORT

        for track in tracks: # Проходит по каждому треку
            if not track.is_confirmed(): # Проверяет, подтверждён ли трек
                continue

            x1, y1, x2, y2 = track.to_ltrb() # Получает координаты бокса трека
            frame = blur_rounded(frame, (x1, y1, x2, y2)) # Применяет скруглённый блюр к области лица

        writer.write(frame) # Записывает обработанный кадр в выходное видео

    cap.release() # Освобождает ресурсы захвата видео
    writer.release() # Освобождает ресурсы записи видео
    print("🎉 ГОТОВО! Сохранено:", output_path) 


# =====================================================
# 5. ЗАПУСК
# =====================================================

video_url = "https://www.youtube.com/watch?v=TV0lRfxdsRg"  # Ссылка на YouTube видео
weights_path = "/root/project/runs/detect/train3/weights/best.pt" # Путь к обученной YOLO-face модели

# 1) скачать видео
input_path = download_video(video_url) 

# 2) убедиться, что оно не AV1
input_path = ensure_h264(input_path)

# 3) запустить обработку
process_video(input_path, "output_blurred_stable.mp4", weights_path)


📥 Скачиваю видео с YouTube...
[youtube] Extracting URL: https://www.youtube.com/watch?v=TV0lRfxdsRg
[youtube] TV0lRfxdsRg: Downloading webpage


[youtube] TV0lRfxdsRg: Downloading android sdkless player API JSON
[youtube] TV0lRfxdsRg: Downloading web safari player API JSON


[youtube] TV0lRfxdsRg: Downloading m3u8 information


[info] TV0lRfxdsRg: Downloading 1 format(s): 300
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 44
[download] Destination: input.mp4
[download] 100% of   41.42MiB in 00:00:01 at 24.93MiB/s                  
[FixupM3u8] Fixing MPEG-TS in MP4 container of "input.mp4"
✔ Видео скачано: input.mp4
🎥 Кодек обнаружен: h264
🚀 Запускаю обработку видео...


/opt/miniforge3/lib/python3.12/site-packages/deep_sort_realtime/embedder/embedder_pytorch.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources



0: 384x640 22 faces, 5.6ms
Speed: 1.3ms preprocess, 5.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 faces, 5.3ms
Speed: 0.6ms preprocess, 5.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 faces, 5.4ms
Speed: 0.6ms preprocess, 5.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 faces, 5.2ms
Speed: 0.5ms preprocess, 5.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 34 faces, 5.3ms
Speed: 0.6ms preprocess, 5.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 29 faces, 5.2ms
Speed: 0.6ms preprocess, 5.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 26 faces, 5.2ms
Speed: 0.6ms preprocess, 5.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 faces, 5.2ms
Speed: 0.6ms preprocess, 5.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x

In [ ]:
# Простая детекция лиц на изображении с помощью YOLO-face, смотрим как модель находит лица
import os
import cv2
import urllib.request
from ultralytics import YOLO

# =========================
# НАСТРОЙКИ
# =========================
WEIGHTS = "/root/project/runs/detect/train3/weights/best.pt"  # путь к твоей обученной модели
TARGET_URL = "https://ichef.bbci.co.uk/ace/standard/976/cpsprodpb/b7b3/live/66207fb0-bfbf-11f0-8db2-d92fbd46f955.jpg.webp"
TARGET_PATH = "target_face.jpg"

OUT_IMG = "target_detected.jpg"
CONF = 0.25        # порог confidence (попробуй 0.10 если не находит)
IOU = 0.5          # NMS iou

# =========================
# СКАЧАТЬ ФОТО (если надо)
# =========================
if TARGET_URL and not os.path.exists(TARGET_PATH): # если файла нет, скачиваем
    print("📥 Download target image...") 
    urllib.request.urlretrieve(TARGET_URL, TARGET_PATH) # скачиваем файл
    print("✔ Saved:", TARGET_PATH)

# =========================
# ЗАГРУЗКА МОДЕЛИ
# =========================
print("🚀 Loading YOLO:", WEIGHTS)
model = YOLO(WEIGHTS) # загружаем модель

# =========================
# ЗАГРУЗКА И ПРОГОН
# =========================
img = cv2.imread(TARGET_PATH) # читаем изображение
if img is None:
    raise RuntimeError(f"❌ Can't read image: {TARGET_PATH}") # ошибка если не удалось прочитать

results = model.predict(img, conf=CONF, iou=IOU, verbose=False)[0] # прогоняем модель

boxes = results.boxes.xyxy.cpu().numpy() if results.boxes is not None else [] # получаем bbox
confs = results.boxes.conf.cpu().numpy() if results.boxes is not None else [] # получаем confidence

print(f"👤 Found faces: {len(boxes)}")
annot = img.copy()

# =========================
# РИСУЕМ BBOX
# =========================
for i, (box, c) in enumerate(zip(boxes, confs), start=1): # перебираем все боксы и confidence
    x1, y1, x2, y2 = map(int, box) # получаем координаты бокса
    cv2.rectangle(annot, (x1, y1), (x2, y2), (0, 255, 0), 2) # рисуем bbox
    cv2.putText(
        annot,
        f"{i}: {c:.2f}", # текст с номером лица и confidence
        (x1, max(0, y1 - 8)), # позиция текста
        cv2.FONT_HERSHEY_SIMPLEX, # шрифт
        0.7, # размер шрифта
        (0, 255, 0), # цвет текста
        2
    )

# =========================
# СОХРАНЯЕМ РЕЗУЛЬТАТ
# =========================
cv2.imwrite(OUT_IMG, annot)
print("🖼 Saved annotated image:", OUT_IMG)

# (опционально) сохранить каждый кроп лица отдельно
os.makedirs("crops", exist_ok=True)
for i, box in enumerate(boxes, start=1):
    x1, y1, x2, y2 = map(int, box)
    crop = img[max(0,y1):max(0,y2), max(0,x1):max(0,x2)]
    if crop.size > 0:
        cv2.imwrite(f"crops/face_{i:02d}.jpg", crop)
print("📁 Crops saved to: crops/")


🚀 Loading YOLO: /root/project/runs/detect/train3/weights/best.pt
👤 Found faces: 1
🖼 Saved annotated image: target_detected.jpg
📁 Crops saved to: crops/


In [ ]:
# Простая детекция лиц на групповом изображении с помощью YOLO-face, кропы лиц которые находит модель
import os
import cv2
import urllib.request
from ultralytics import YOLO

# =========================
# НАСТРОЙКИ
# =========================
WEIGHTS = "/root/project/runs/detect/train3/weights/best.pt"  # путь к твоей обученной модели
TARGET_URL = "https://c8.alamy.com/comp/2RR3BG4/inter-miamis-lionel-messi-10-and-teammates-pose-for-group-photos-prior-to-an-mls-soccer-match-against-the-los-angeles-fc-inter-miami-cf-31-los-angeles-fc-2RR3BG4.jpg"  
TARGET_PATH = "target_face_group.jpg"

OUT_IMG = "target_detected_group.jpg"
CONF = 0.25        # порог confidence (попробуй 0.10 если не находит)
IOU = 0.5          # iou ддля NMS что бы не дублировало боксы

# =========================
# СКАЧАТЬ ФОТО (если надо)
# =========================
if TARGET_URL and not os.path.exists(TARGET_PATH):
    print("📥 Download target image...")
    urllib.request.urlretrieve(TARGET_URL, TARGET_PATH)
    print("✔ Saved:", TARGET_PATH)

# =========================
# ЗАГРУЗКА МОДЕЛИ
# =========================
print("🚀 Loading YOLO:", WEIGHTS)
model = YOLO(WEIGHTS)

# =========================
# ЗАГРУЗКА И ПРОГОН
# =========================
img = cv2.imread(TARGET_PATH) # читаем изображение
if img is None:
    raise RuntimeError(f"❌ Can't read image: {TARGET_PATH}") # ошибка если не удалось прочитать

results = model.predict(img, conf=CONF, iou=IOU, verbose=False)[0] # прогоняем модель

boxes = results.boxes.xyxy.cpu().numpy() if results.boxes is not None else [] # получаем bbox
confs = results.boxes.conf.cpu().numpy() if results.boxes is not None else [] # получаем confidence

print(f"👤 Found faces: {len(boxes)}")
annot = img.copy()

# =========================
# РИСУЕМ BBOX
# =========================
for i, (box, c) in enumerate(zip(boxes, confs), start=1): # перебираем все боксы и confidence
    x1, y1, x2, y2 = map(int, box) # получаем координаты бокса
    cv2.rectangle(annot, (x1, y1), (x2, y2), (0, 255, 0), 2) # рисуем bbox
    cv2.putText(
        annot,
        f"{i}: {c:.2f}", # текст с номером лица и confidence
        (x1, max(0, y1 - 8)), # позиция текста
        cv2.FONT_HERSHEY_SIMPLEX, # шрифт
        0.7, # размер шрифта
        (0, 255, 0), # цвет текста
        2
    )

# =========================
# СОХРАНЯЕМ РЕЗУЛЬТАТ
# =========================
cv2.imwrite(OUT_IMG, annot) # сохраняем аннотированное изображение
print("🖼 Saved annotated image:", OUT_IMG) 

# (опционально) сохранить каждый кроп лица отдельно
os.makedirs("crops_group", exist_ok=True) # создаём папку для кропов
for i, box in enumerate(boxes, start=1): # перебираем все боксы
    x1, y1, x2, y2 = map(int, box) # получаем координаты бокса
    crop = img[max(0,y1):max(0,y2), max(0,x1):max(0,x2)] # вырезаем кроп лица
    if crop.size > 0:
        cv2.imwrite(f"crops_group/face_{i:02d}.jpg", crop) # сохраняем кроп
print("📁 Crops saved to: crops_group/")


📥 Download target image...
✔ Saved: target_face_group.jpg
🚀 Loading YOLO: /root/project/runs/detect/train3/weights/best.pt
👤 Found faces: 11
🖼 Saved annotated image: target_detected_group.jpg
📁 Crops saved to: crops_group/


In [19]:
# Эмбеддинги целевого лица с помощью YOLO-face и ArcFace
import os
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from insightface.model_zoo import get_model

# =========================
# 1️⃣ НАСТРОЙКИ
# =========================

IMG_PATH = "target_face.jpg"   # целевое фото
OUT_DIR = "arcface_crops_embeddings" # папка для кропов лиц и эмбеддингов
os.makedirs(OUT_DIR, exist_ok=True) # создаём папку если нет

YOLO_WEIGHTS = "/root/project/runs/detect/train3/weights/best.pt" # путь к твоей обученной модели YOLO-face

PAD_RATIO = 0.4          # padding вокруг лица (40%)
ARC_SIZE = 112           # вход ArcFace, обычно 112x112

# =========================
# 2️⃣ ЗАГРУЗКА МОДЕЛЕЙ
# =========================

print("🚀 Loading YOLO-face...")
yolo = YOLO(YOLO_WEIGHTS)

print("🚀 Loading ArcFace...")
from insightface.app import FaceAnalysis

print("🚀 Loading InsightFace (buffalo_l)...")

face_app = FaceAnalysis( 
    name="buffalo_l", # модель ArcFace
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"] # использовать GPU если есть
)

face_app.prepare(
    ctx_id=0 if torch.cuda.is_available() else -1, # GPU или CPU
    det_size=(640, 640) # размер для детекции лиц
)

# =========================
# 3️⃣ ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# =========================

def l2norm(x): # L2 нормализация эмбеддинга, что была единичная длина
    return x / np.linalg.norm(x) 

def crop_with_padding(img, box, pad_ratio): # кроп с паддингом
    h, w = img.shape[:2] # размеры изображения
    x1, y1, x2, y2 = map(int, box) # координаты бокса

    bw = x2 - x1 # ширина бокса
    bh = y2 - y1 # высота бокса

    pad_w = int(bw * pad_ratio) # паддинг по ширине
    pad_h = int(bh * pad_ratio) # паддинг по высоте

    x1p = max(0, x1 - pad_w) # левая с учётом паддинга
    y1p = max(0, y1 - pad_h) # верхняя с учётом паддинга
    x2p = min(w, x2 + pad_w) # правая с учётом паддинга
    y2p = min(h, y2 + pad_h) # нижняя с учётом паддинга

    return img[y1p:y2p, x1p:x2p] # вырезанный кроп с паддингом

# Делаем изображение квадратным и добавляем черные поля для того чтобы не искажать лицо при ресайзе
def make_square(img): # делаем изображение квадратным, добавляя чёрные поля
    h, w = img.shape[:2] # размеры изображения
    size = max(h, w) # размер стороны квадрата

    square = np.zeros((size, size, 3), dtype=np.uint8) # чёрный квадрат
    y = (size - h) // 2 # вертикальная позиция для вставки
    x = (size - w) // 2 # горизонтальная позиция для вставки

    square[y:y+h, x:x+w] = img # вставляем оригинал в центр квадрата
    return square

# =========================
# 4️⃣ ЗАГРУЗКА ИЗОБРАЖЕНИЯ
# =========================

img = cv2.imread(IMG_PATH) # читаем изображение
if img is None:
    raise RuntimeError("❌ Не удалось загрузить изображение")

annot = img.copy() # копия для аннотаций

# =========================
# 5️⃣ YOLO → DETECT
# =========================

results = yolo(img, conf=0.25, device='cpu', verbose=False)[0] # прогоняем YOLO

if results.boxes is None: # если нет боксов
    print("❌ YOLO не нашёл лиц")
    exit()

boxes = results.boxes.xyxy.cpu().numpy() # получаем bbox лиц
print(f"👤 Найдено лиц: {len(boxes)}")

# =========================
# 6️⃣ YOLO → ArcFace
# =========================

embeddings = [] # список для эмбеддингов

for i, box in enumerate(boxes, start=1): # перебираем все боксы лиц

    # 🔹 padding
    face = crop_with_padding(img, box, PAD_RATIO)  # кроп с паддингом
    if face.size == 0:
        continue

    # 🔹 square 
    face = make_square(face) # делаем квадратным

    # 🔹 resize под ArcFace
    face = cv2.resize(face, (ARC_SIZE, ARC_SIZE)) # ресайз под ArcFace

    # 🔹 BGR → RGB
    face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB) # BGR → RGB

    # 🔹 ArcFace embedding
    faces = face_app.get(face_rgb) # получаем лица через ArcFace
    if len(faces) > 0:
        emb = faces[0].embedding # берём эмбеддинг первого лица
        emb = l2norm(emb) # L2 нормализация
        embeddings.append(emb) # сохраняем эмбеддинг

    # 🔹 сохраняем кроп
    cv2.imwrite(f"{OUT_DIR}/face_{i:02d}.jpg", face) # сохраняем кроп лица

    # 🔹 рисуем bbox
    x1, y1, x2, y2 = map(int, box) # координаты бокса
    cv2.rectangle(annot, (x1, y1), (x2, y2), (0,255,0), 2) # рисуем прямоугольник
    cv2.putText(
        annot, # текст с номером лица
        f"{i}", # текст с номером лица
        (x1, y1 - 5), # позиция текста
        cv2.FONT_HERSHEY_SIMPLEX, # шрифт
        0.7, # размер шрифта
        (0,255,0), # цвет текста
        2 # толщина текста
    )

print(f"✅ ArcFace embeddings получены: {len(embeddings)}")

# сохраняем первый эмбеддинг как целевой 
if len(embeddings) > 0: # если есть эмбеддинги
    target_embedding = embeddings[0] # первый эмбеддинг
    np.save("target_embedding.npy", target_embedding) # сохраняем в файл
    print("✅ Целевой эмбеддинг сохранён: target_embedding.npy") 
else:
    print("⚠️ Нет эмбеддингов для сохранения")


# =========================
# 7️⃣ СОХРАНЕНИЕ РЕЗУЛЬТАТА
# =========================

cv2.imwrite("arcface_target_detected.jpg", annot)
print("🖼 Результат сохранён: arcface_target_detected.jpg")
print("📁 Кропы сохранены в:", OUT_DIR)


🚀 Loading YOLO-face...
🚀 Loading ArcFace...
🚀 Loading InsightFace (buffalo_l)...
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0

In [18]:
# Эмбединги лиц с помощью YOLO-face и ArcFace
import os
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from insightface.model_zoo import get_model

# =========================
# 1️⃣ НАСТРОЙКИ
# =========================

IMG_PATH = "target_face_group.jpg"   # групповое фото
OUT_DIR = "arcface_crops" # папка для кропов лиц и эмбеддингов
os.makedirs(OUT_DIR, exist_ok=True) # создаём папку если нет

YOLO_WEIGHTS = "/root/project/runs/detect/train3/weights/best.pt"

PAD_RATIO = 0.4          # padding вокруг лица (40%)
ARC_SIZE = 112           # вход ArcFace

# =========================
# 2️⃣ ЗАГРУЗКА МОДЕЛЕЙ
# =========================

print("🚀 Loading YOLO-face...")
yolo = YOLO(YOLO_WEIGHTS)

print("🚀 Loading ArcFace...")
from insightface.app import FaceAnalysis

print("🚀 Loading InsightFace (buffalo_l)...")

face_app = FaceAnalysis(
    name="buffalo_l",
    providers=["CPUExecutionProvider"]
)

face_app.prepare(
    ctx_id=-1,
    det_size=(640, 640)
)

# =========================
# 3️⃣ ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# =========================

def l2norm(x): # L2 нормализация эмбеддинга
    return x / np.linalg.norm(x)

def crop_with_padding(img, box, pad_ratio): # кроп с паддингом
    h, w = img.shape[:2]
    x1, y1, x2, y2 = map(int, box)

    bw = x2 - x1 # ширина бокса
    bh = y2 - y1 # высота бокса

    pad_w = int(bw * pad_ratio) # паддинг по ширине
    pad_h = int(bh * pad_ratio) # паддинг по высоте

    x1p = max(0, x1 - pad_w) # левая с учётом паддинга
    y1p = max(0, y1 - pad_h) # верхняя с учётом паддинга
    x2p = min(w, x2 + pad_w) # правая с учётом паддинга
    y2p = min(h, y2 + pad_h) # нижняя с учётом паддинга

    return img[y1p:y2p, x1p:x2p]

def make_square(img): # делаем изображение квадратным, добавляя чёрные поля
    h, w = img.shape[:2] # размеры изображения
    size = max(h, w) # размер стороны квадрата

    square = np.zeros((size, size, 3), dtype=np.uint8) # чёрный квадрат
    y = (size - h) // 2 # вертикальная позиция для вставки
    x = (size - w) // 2 # горизонтальная позиция для вставки

    square[y:y+h, x:x+w] = img # вставляем оригинал в центр квадрата
    return square

# =========================
# 4️⃣ ЗАГРУЗКА ИЗОБРАЖЕНИЯ
# =========================

img = cv2.imread(IMG_PATH) # читаем изображение
if img is None:
    raise RuntimeError("❌ Не удалось загрузить изображение")

annot = img.copy() # копия для аннотаций

# =========================
# 5️⃣ YOLO → DETECT
# =========================

results = yolo(img, conf=0.25, device='cpu', verbose=False)[0] # прогоняем YOLO

if results.boxes is None:  # если нет боксов
    print("❌ YOLO не нашёл лиц") 
    exit()

boxes = results.boxes.xyxy.cpu().numpy() # получаем bbox лиц
print(f"👤 Найдено лиц: {len(boxes)}")

# =========================
# 6️⃣ YOLO → ArcFace
# =========================

embeddings = [] # список для эмбеддингов

for i, box in enumerate(boxes, start=1): # перебираем все боксы лиц

    # 🔹 padding
    face = crop_with_padding(img, box, PAD_RATIO) # кроп с паддингом
    if face.size == 0: 
        continue

    # 🔹 square
    face = make_square(face) # делаем квадратным

    # 🔹 resize под ArcFace
    face = cv2.resize(face, (ARC_SIZE, ARC_SIZE)) # ресайз под ArcFace

    # 🔹 BGR → RGB
    face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB) # BGR → RGB

    # 🔹 ArcFace embedding
    faces = face_app.get(face_rgb) # получаем лица через ArcFace
    if len(faces) > 0: 
        emb = faces[0].embedding # берём эмбеддинг первого лица
        emb = l2norm(emb) # L2 нормализация
        embeddings.append(emb) # сохраняем эмбеддинг

    # 🔹 сохраняем кроп
    cv2.imwrite(f"{OUT_DIR}/face_{i:02d}.jpg", face)

    # 🔹 рисуем bbox
    x1, y1, x2, y2 = map(int, box) # координаты бокса
    cv2.rectangle(annot, (x1, y1), (x2, y2), (0,255,0), 2) # рисуем прямоугольник
    cv2.putText( 
        annot, # текст с номером лица
        f"{i}", # название лица
        (x1, y1 - 5), # позиция текста
        cv2.FONT_HERSHEY_SIMPLEX, # шрифт
        0.7, # размер шрифта
        (0,255,0), # цвет текста
        2 # толщина текста
    )

print(f"✅ ArcFace embeddings получены: {len(embeddings)}")

# =========================
# 8️⃣ ЗАГРУЗКА ЦЕЛЕВОГО ЭМБЕДДИНГА
# =========================

target_emb = np.load("target_embedding.npy") # загружаем целевой эмбеддинг
target_emb = target_emb / np.linalg.norm(target_emb) # L2 нормализация

print("\n🎯 Целевой эмбеддинг загружен")

# =========================
# 9️⃣ СРАВНЕНИЕ (Cosine Similarity → %)
# =========================

def cosine_similarity(a, b): # косинусная схожесть
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

print("\n🔍 Сравнение с целевым лицом:\n")

results = [] # список для результатов

for idx, emb in enumerate(embeddings, start=1): # перебираем эмбеддинги лиц
    emb = emb / np.linalg.norm(emb) # L2 нормализация

    cos_sim = cosine_similarity(target_emb, emb) # косинусная схожесть

    # перевод в проценты (0–100) 
    similarity_percent = (cos_sim + 1) / 2 * 100

    results.append((idx, similarity_percent)) # сохраняем результат

    print(f"face_{idx:02d}  →  {similarity_percent:.2f}%") 

# =========================
# 🔟 ТОП ПОХОЖИХ
# =========================

results.sort(key=lambda x: x[1], reverse=True) # сортируем по убыванию схожести

print("\n🏆 Самые похожие лица:")
for idx, score in results: # перебираем результаты
    print(f"face_{idx:02d}  →  {score:.2f}%")


# =========================
# 7️⃣ СОХРАНЕНИЕ РЕЗУЛЬТАТА
# =========================

cv2.imwrite("arcface_group_detected.jpg", annot)
print("🖼 Результат сохранён: arcface_group_detected.jpg")
print("📁 Кропы сохранены в:", OUT_DIR)


🚀 Loading YOLO-face...
🚀 Loading ArcFace...
🚀 Loading InsightFace (buffalo_l)...
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx re

In [15]:
# =========================
# 🔟 ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ
# =========================

VIS_IMG = img.copy()

THRESHOLD_SAME = 75.0  # порог "ТОЧНО ТОТ"
THRESHOLD_MAYBE = 50.0  # порог "ВОЗМОЖНО ТОТ"

for i, box in enumerate(boxes): # перебираем все боксы лиц
    if i >= len(embeddings): # если нет эмбеддинга — пропускаем
        continue

    emb = embeddings[i] # получаем эмбеддинг
    emb = emb / np.linalg.norm(emb) # L2 нормализация

    cos_sim = np.dot(target_emb, emb) # косинусная схожесть
    similarity_percent = (cos_sim + 1) / 2 * 100 # перевод в проценты

    # 🎨 цвет рамки
    if similarity_percent >= THRESHOLD_SAME:
        color = (0, 255, 0)       # зелёный
        label = f"{similarity_percent:.1f}%"
    elif similarity_percent >= THRESHOLD_MAYBE:
        color = (0, 255, 255)     # жёлтый
        label = f"{similarity_percent:.1f}%"
    else:
        color = (0, 0, 255)       # красный
        label = f"{similarity_percent:.1f}%"

    x1, y1, x2, y2 = map(int, box) # координаты бокса

    cv2.rectangle(VIS_IMG, (x1, y1), (x2, y2), color, 2) # рисуем прямоугольник
    cv2.putText( 
        VIS_IMG, # изображение для аннотаций
        label, # текст с процентами схожести
        (x1, max(20, y1 - 10)), # позиция текста
        cv2.FONT_HERSHEY_SIMPLEX, # шрифт
        0.6,
        color,
        2
    )

# =========================
# 💾 СОХРАНЕНИЕ
# =========================

OUT_VIS = "face_verification_result.jpg" # файл для сохранения
cv2.imwrite(OUT_VIS, VIS_IMG)

print(f"🖼 Результат сохранён: {OUT_VIS}")


🖼 Результат сохранён: face_verification_result.jpg


In [ ]:
# сравнение лиц на изображении с  только помощью InsightFace
import cv2
import numpy as np
import os
from insightface.app import FaceAnalysis

# =========================
# 1️⃣ НАСТРОЙКИ
# =========================
IMAGE_PATH = "target_face_group.jpg"      # изображение для поиска
TARGET_PATH = "target_face.jpg"    # целевое лицо
OUTPUT_PATH = "result.jpg"            # куда сохранить результат

THRESHOLD = 0.45              # порог похожести
DET_SIZE = (640, 640)         # размер для детекции лиц

# =========================
# 2️⃣ ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# =========================
def cosine_similarity(a, b): # косинусная схожесть
    return np.dot(a, b)

# =========================
# 3️⃣ ЗАГРУЗКА INSIGHTFACE
# =========================
print("🚀 Loading InsightFace (buffalo_l)...") 

app = FaceAnalysis( 
    name="buffalo_l", # модель ArcFace
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"]
)

app.prepare(ctx_id=0, det_size=DET_SIZE) # использовать GPU если есть

# =========================
# 4️⃣ ЦЕЛЕВОЕ ЛИЦО → EMBEDDING
# =========================
target_img = cv2.imread(TARGET_PATH) # читаем целевое изображение
if target_img is None:
    raise RuntimeError("❌ Не удалось загрузить target изображение")

target_faces = app.get(target_img) # получаем лица на целевом изображении

if len(target_faces) == 0: # если нет лиц
    raise RuntimeError("❌ Лицо на target изображении не найдено")

target_face = target_faces[0] # берём первое лицо
target_emb = target_face.normed_embedding # нормализованный эмбеддинг

print("🎯 Target embedding готов")

# =========================
# 5️⃣ ПОИСК ЛИЦ НА ИЗОБРАЖЕНИИ
# =========================
img = cv2.imread(IMAGE_PATH) # читаем групповое изображение
if img is None:
    raise RuntimeError("❌ Не удалось загрузить изображение")

faces = app.get(img) # получаем лица на групповом изображении
print(f"👤 Найдено лиц: {len(faces)}")

# =========================
# 6️⃣ СРАВНЕНИЕ
# =========================
results = [] # список для результатов

for i, face in enumerate(faces): # перебираем все найденные лица
    emb = face.normed_embedding # нормализованный эмбеддинг
    sim = cosine_similarity(target_emb, emb) # косинусная схожесть

    results.append({ 
        "id": i, # номер лица
        "bbox": face.bbox.astype(int), # координаты бокса
        "similarity": float(sim) # похожесть
    })

# сортировка по похожести
results = sorted(results, key=lambda x: x["similarity"], reverse=True) # сортируем по убыванию похожести

# =========================
# 7️⃣ ВИЗУАЛИЗАЦИЯ
# =========================
for r in results: # перебираем результаты
    if r["similarity"] < THRESHOLD: # если ниже порога — пропускаем
        continue

    x1, y1, x2, y2 = r["bbox"] # координаты бокса
    sim = r["similarity"] # похожесть

    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2) # рисуем прямоугольник
    cv2.putText(
        img, # изображение для аннотаций
        f"{sim:.2f}", # текст с похожестью
        (x1, y1 - 10), # позиция текста
        cv2.FONT_HERSHEY_SIMPLEX, # шрифт
        0.6, # размер шрифта
        (0, 255, 0), # цвет текста
        2 # толщина текста
    )

# =========================
# 8️⃣ СОХРАНЕНИЕ
# =========================
cv2.imwrite(OUTPUT_PATH, img)
print(f"🖼 Результат сохранён: {OUTPUT_PATH}")


🚀 Loading InsightFace (buffalo_l)...
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvi

In [4]:
# сравнение лиц на изображении с процентами похожести с помощью InsightFace ,где помечены все лицас процентом поховести

import cv2
import numpy as np
from insightface.app import FaceAnalysis

# =========================
# 1️⃣ НАСТРОЙКИ
# =========================
IMAGE_PATH = "target_face_group.jpg"      # групповое фото
TARGET_PATH = "target_face.jpg"    # целевое лицо
OUTPUT_PATH = "result_percent.jpg" #  куда сохранить результат

DET_SIZE = (640, 640)

# =========================
# 2️⃣ ФУНКЦИИ
# =========================
def cosine_similarity(a, b):  #  косинусная схожесть
    return np.dot(a, b)

def sim_to_percent(sim): # перевод cosine similarity в проценты
    """
    Перевод cosine similarity в %
    typical range ArcFace: [0.2 .. 0.8]
    """
    sim = np.clip(sim, 0.0, 1.0) # ограничиваем от 0 до 1, чтобы не было отрицательных значений
    return int(sim * 100) # перевод в проценты

# =========================
# 3️⃣ ЗАГРУЗКА INSIGHTFACE
# =========================
print("🚀 Loading InsightFace (buffalo_l)...")

app = FaceAnalysis(
    name="buffalo_l", # модель ArcFace
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"]
)
app.prepare(ctx_id=0, det_size=DET_SIZE) # использовать GPU если есть

# =========================
# 4️⃣ ЦЕЛЕВОЕ ЛИЦО
# =========================
target_img = cv2.imread(TARGET_PATH) # читаем целевое изображение
if target_img is None: # если не удалось прочитать
    raise RuntimeError("❌ Target image not found")

target_faces = app.get(target_img) # получаем лица на целевом изображении
if len(target_faces) == 0: # если нет лиц
    raise RuntimeError("❌ Лицо на target не найдено")

target_emb = target_faces[0].normed_embedding
print("🎯 Target embedding готов")

# =========================
# 5️⃣ ПОИСК ЛИЦ
# =========================
img = cv2.imread(IMAGE_PATH) # читаем групповое изображение
if img is None: # если не удалось прочитать
    raise RuntimeError("❌ Image not found")

faces = app.get(img) # получаем лица на групповом изображении
print(f"👤 Найдено лиц: {len(faces)}")

# =========================
# 6️⃣ СРАВНЕНИЕ + ВИЗУАЛИЗАЦИЯ
# =========================
for i, face in enumerate(faces, start=1): # перебираем все найденные лица
    emb = face.normed_embedding # нормализованный эмбеддинг
    sim = cosine_similarity(target_emb, emb) # косинусная схожесть
    percent = sim_to_percent(sim) # перевод в проценты

    x1, y1, x2, y2 = face.bbox.astype(int) # координаты бокса

    # Цвет: зелёный → высокий %, красный → низкий
    color = (
        int(255 * (1 - percent / 100)),
        int(255 * (percent / 100)),
        0
    )

    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2) # рисуем прямоугольник

    cv2.putText(
        img, # изображение для аннотаций
        f"{percent}%", # текст с процентом похожести
        (x1, y1 - 8), # позиция текста
        cv2.FONT_HERSHEY_SIMPLEX, # шрифт
        0.75, # размер шрифта
        color, # цвет текста
        3 # толщина текста
    )

# =========================
# 7️⃣ СОХРАНЕНИЕ
# =========================
cv2.imwrite(OUTPUT_PATH, img)
print(f"🖼 Результат сохранён: {OUTPUT_PATH}")


🚀 Loading InsightFace (buffalo_l)...
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvi

In [51]:
%pip install ultralytics insightface facenet-pytorch onnxruntime-gpu


INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 95.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 97.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 58.3 MB/s  0:00:11m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 57.2 MB/s  0:00:06m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.6 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 54.6 MB/s  0:00:00m0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# =========================
# ИМПОРТЫ
# =========================

import os                      # Работа с файлами и путями
import cv2                     # OpenCV: изображения, видео, отрисовка
import numpy as np              # Численные операции
import urllib.request           # Скачивание файлов по URL

from insightface.app import FaceAnalysis  # ArcFace + детекция лиц


# =========================
# 1) НАСТРОЙКИ
# =========================

# URL изображения с ЦЕЛЕВЫМ лицом (эталон)
TARGET_URL = "https://ichef.bbci.co.uk/ace/standard/976/cpsprodpb/b7b3/live/66207fb0-bfbf-11f0-8db2-d92fbd46f955.jpg.webp"

# Локальный путь для сохранения эталонного лица
TARGET_PATH = "target_face.jpg"

# URL фотографии, на которой нужно искать людей
# (если None — используется локальный файл)
PHOTO_URL = "https://c8.alamy.com/comp/2RR3BG4/inter-miamis-lionel-messi-10-and-teammates-pose-for-group-photos-prior-to-an-mls-soccer-match-against-the-los-angeles-fc-inter-miami-cf-31-los-angeles-fc-2RR3BG4.jpg"  

# Локальный путь к фото для анализа
PHOTO_PATH = "photo.jpg"

# Файл с результатом (размеченное фото)
OUT_ANNOTATED = "result_annotated.jpg"

# Размер входа детектора InsightFace
# Чем больше — тем лучше мелкие лица, но медленнее
DET_SIZE = (640, 640)

# Порог "совпадения" (для подписи match)
# Проценты считаются всегда, порог только для пометки
THRESHOLD_SIM = 0.35


# =========================
# 2) ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# =========================

def download(url: str, path: str):
    """
    Скачивает файл по URL, если он ещё не существует
    """
    if url is None:             # Если URL не задан — ничего не делаем
        return
    if os.path.exists(path):    # Если файл уже есть — не скачиваем
        print(f"✓ Уже есть: {path}")
        return
    print(f"📥 Скачиваю: {url}")
    urllib.request.urlretrieve(url, path)  # Скачивание
    print(f"✔ Сохранено: {path}")


def l2norm(x: np.ndarray):
    """
    L2-нормализация вектора (обязательно для cosine similarity)
    """
    n = np.linalg.norm(x)        # Длина вектора
    if n < 1e-12:                # Защита от деления на ноль
        return None
    return x / n                 # Нормированный embedding


def cosine(a: np.ndarray, b: np.ndarray) -> float: # косинусное сходство
    return float(np.dot(a, b))


def sim_to_percent(sim: float) -> float: # перевод cosine similarity в проценты
    sim01 = max(0.0, min(1.0, sim))  # Ограничиваем диапазон
    return sim01 * 100.0


# =========================
# 3) ИНИЦИАЛИЗАЦИЯ INSIGHTFACE
# =========================

print("🧠 Загружаю InsightFace (buffalo_l)...")

# Создаём объект InsightFace
# buffalo_l = ArcFace + детектор лиц
app = FaceAnalysis(
    name="buffalo_l",
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"]  # GPU → CPU fallback
)

# Подготовка модели (размер входа детектора)
app.prepare(ctx_id=0, det_size=DET_SIZE)

print("✔ InsightFace готов")


# =========================
# 4) ЗАГРУЗКА ИЗОБРАЖЕНИЙ
# =========================

# Скачиваем эталонное лицо
download(TARGET_URL, TARGET_PATH)

# Если фото задано по URL — скачиваем
if PHOTO_URL is not None:
    download(PHOTO_URL, PHOTO_PATH)

# Загружаем эталонное изображение
target_img = cv2.imread(TARGET_PATH)
if target_img is None:
    raise RuntimeError(f"❌ Не удалось прочитать target image: {TARGET_PATH}")

# Загружаем фото для анализа
photo = cv2.imread(PHOTO_PATH)
if photo is None:
    raise RuntimeError(
        f"❌ Не удалось прочитать фото для анализа: {PHOTO_PATH}\n"
        f"Если фото по URL — задай PHOTO_URL. Если локально — проверь путь PHOTO_PATH."
    )


# =========================
# 5) ЭМБЕДДИНГ ЦЕЛЕВОГО ЛИЦА
# =========================

# InsightFace работает в RGB
target_rgb = cv2.cvtColor(target_img, cv2.COLOR_BGR2RGB)

# Детектируем лица на эталонном фото
target_faces = app.get(target_rgb)

if len(target_faces) == 0:
    raise RuntimeError("❌ На target-фото не найдено лицо.")

# Если найдено несколько лиц — берём САМОЕ БОЛЬШОЕ
target_faces = sorted(
    target_faces,
    key=lambda f: (f.bbox[2] - f.bbox[0]) * (f.bbox[3] - f.bbox[1]),
    reverse=True
)

# Получаем embedding целевого лица и нормализуем
target_emb = l2norm(target_faces[0].embedding.astype("float32"))

if target_emb is None:
    raise RuntimeError("❌ Не удалось нормализовать embedding.")

print("🎯 Target embedding готов")


# =========================
# 6) АНАЛИЗ ФОТО
# =========================

# Переводим фото в RGB
photo_rgb = cv2.cvtColor(photo, cv2.COLOR_BGR2RGB)

# Детектируем лица на фото
faces = app.get(photo_rgb)

print(f"👤 Найдено лиц на фото: {len(faces)}")

# Копия изображения для отрисовки
annot = photo.copy()

# Список результатов для сортировки
results = []

# Проходим по всем найденным лицам
for idx, f in enumerate(faces, start=1):

    # Нормализуем embedding лица
    emb = l2norm(f.embedding.astype("float32"))
    if emb is None:
        continue

    # Считаем сходство с целевым лицом
    sim = cosine(target_emb, emb)
    pct = sim_to_percent(sim)

    # Bounding box лица
    x1, y1, x2, y2 = map(int, f.bbox.tolist())

    # Сохраняем результат
    results.append((idx, pct, sim, (x1, y1, x2, y2)))

    # Рисуем прямоугольник
    cv2.rectangle(annot, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Текст с процентом
    label = f"{pct:.1f}%"
    if sim >= THRESHOLD_SIM:
        label += " (match)"

    cv2.putText(
        annot,
        label,
        (x1, max(0, y1 - 10)),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,
        (0, 255, 0),
        2
    )


# =========================
# 7) СОХРАНЕНИЕ И ВЫВОД
# =========================

# Сохраняем размеченное изображение
cv2.imwrite(OUT_ANNOTATED, annot)
print(f"🖼 Сохранено: {OUT_ANNOTATED}")

# Сортируем лица по убыванию похожести
results.sort(key=lambda x: x[1], reverse=True)

# Печатаем рейтинг
print("\nРейтинг похожести:")
for idx, pct, sim, bbox in results:
    print(f"#{idx:02d}: {pct:6.2f}% | cosine={sim:.3f} | bbox={bbox}")


🧠 Загружаю InsightFace (buffalo_l)...
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProv

In [27]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from insightface.app import FaceAnalysis

# =========================
# НАСТРОЙКИ
# =========================
VIDEO_PATH = "input.mp4"              # путь к входному видео
OUT_VIDEO = "output_marked.mp4"       # куда сохраняем обработанное видео
TARGET_EMB_PATH = "target_embedding.npy"  # файл с целевым эмбеддингом

YOLO_WEIGHTS = "/root/project/runs/detect/train3/weights/best.pt"  # веса YOLO

PAD_RATIO = 0.4     # сколько пикселей добавляем вокруг лица
ARC_SIZE = 112      # размер кропа для ArcFace (112x112)
THRESHOLD = 75.0    # минимальный процент похожести для отметки
FRAME_SKIP = 2      # пропуск кадров для ускорения (каждый 2-й кадр)

# =========================
# УСТРОЙСТВО
# =========================
# Проверяем, есть ли GPU и задаем устройство для ArcFace
device = 0 if torch.cuda.is_available() else "cpu"
print("🔥 ArcFace использует устройство:", device)

# =========================
# ЗАГРУЗКА МОДЕЛЕЙ
# =========================
yolo = YOLO(YOLO_WEIGHTS)  # YOLO-face (детекция лиц) на CPU

# ArcFace через InsightFace
# providers = CPU если GPU недоступен, иначе CUDA
face_app = FaceAnalysis(
    name="buffalo_l",
    providers=["CPUExecutionProvider"] if device == "cpu" else ["CUDAExecutionProvider"]
)
face_app.prepare(
    ctx_id=device if device != "cpu" else -1,  # GPU = device, CPU = -1
    det_size=(640, 640)                        # размер окна детекции
)

# Загружаем целевой эмбеддинг и нормируем его
target_emb = np.load(TARGET_EMB_PATH)
target_emb /= np.linalg.norm(target_emb)

# =========================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# =========================
def crop_with_padding(img, box, pad_ratio):
    """Обрезаем лицо с паддингом вокруг bbox"""
    h, w = img.shape[:2]           # размеры кадра
    x1, y1, x2, y2 = map(int, box) # координаты bbox
    bw, bh = x2 - x1, y2 - y1      # ширина и высота bbox
    pw, ph = int(bw * pad_ratio), int(bh * pad_ratio)  # паддинг
    x1, y1 = max(0, x1 - pw), max(0, y1 - ph)          # границы кадра
    x2, y2 = min(w, x2 + pw), min(h, y2 + ph)
    return img[y1:y2, x1:x2]       # возвращаем кроп

def make_square(img):
    """Делаем квадратный кроп, чтобы ArcFace лучше работал"""
    h, w = img.shape[:2]
    size = max(h, w)
    square = np.zeros((size, size, 3), dtype=np.uint8)  # черный квадрат
    y = (size - h) // 2   # вертикальный сдвиг
    x = (size - w) // 2   # горизонтальный сдвиг
    square[y:y+h, x:x+w] = img  # вставляем лицо в центр
    return square

# =========================
# ОТКРЫТИЕ ВИДЕО
# =========================
cap = cv2.VideoCapture(VIDEO_PATH)  # открываем видео
fps = cap.get(cv2.CAP_PROP_FPS)     # кадры в секунду
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Создаем writer для сохранения видео
writer = cv2.VideoWriter(
    OUT_VIDEO,
    cv2.VideoWriter_fourcc(*"mp4v"),  # кодек
    fps,
    (w, h)
)

print("▶️ Обработка видео...")

frame_id = 0
while True:
    ret, frame = cap.read()       # читаем кадр
    if not ret:
        break                     # если видео закончилось, выходим

    frame_id += 1
    # Пропускаем кадры для ускорения
    if frame_id % FRAME_SKIP != 0:
        writer.write(frame)       # просто пишем кадр без обработки
        continue

    # =========================
    # YOLO (CPU) - детекция лиц
    # =========================
    results = yolo(frame, conf=0.25, device="cpu", verbose=False)[0]

    if results.boxes is None:
        writer.write(frame)       # если лица не найдены, пишем кадр
        continue

    boxes = results.boxes.xyxy.cpu().numpy()  # получаем координаты лиц

    for box in boxes:
        # обрезка лица с паддингом
        face = crop_with_padding(frame, box, PAD_RATIO)
        if face.size == 0:
            continue

        # квадратный кроп + resize для ArcFace
        face = make_square(face)
        face = cv2.resize(face, (ARC_SIZE, ARC_SIZE))
        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

        # =========================
        # ArcFace (GPU если есть)
        # =========================
        faces = face_app.get(face_rgb)  # получаем эмбеддинг
        if len(faces) == 0:
            continue

        emb = faces[0].embedding       # вектор эмбеддинга
        emb /= np.linalg.norm(emb)     # нормируем

        # =========================
        # СРАВНЕНИЕ С ЦЕЛЕВЫМ
        # =========================
        cos_sim = np.dot(target_emb, emb)
        similarity = (cos_sim + 1) / 2 * 100  # конвертируем в %

        # =========================
        # Если похожесть выше порога, рисуем bbox + текст
        # =========================
        if similarity >= THRESHOLD:
            x1, y1, x2, y2 = map(int, box)
            label = f"SAME {similarity:.1f}%"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # зеленый bbox
            cv2.putText(
                frame,
                label,
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2
            )

    # =========================
    # записываем кадр в видео
    # =========================
    writer.write(frame)

# =========================
# ЗАВЕРШЕНИЕ
# =========================
cap.release()
writer.release()
print("✅ Видео сохранено:", OUT_VIDEO)


🔥 ArcFace использует устройство: 0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvide

In [9]:
import cv2                                  # работа с видео/картинками
import numpy as np                          # математика и массивы
import torch                                # проверка GPU
from ultralytics import YOLO                # YOLO для детекции лиц
from insightface.app import FaceAnalysis    # ArcFace/InsightFace для эмбеддингов

# =========================
# 1) НАСТРОЙКИ
# =========================
VIDEO_PATH = "input.mp4"                    # входное видео
OUT_VIDEO = "output_blur_except_target.mp4" # выходное видео
TARGET_EMB_PATH = "target_embedding.npy"    # файл с целевым эмбеддингом

YOLO_WEIGHTS = "/root/project/runs/detect/train3/weights/best.pt"  # веса YOLO-face

PAD_RATIO = 0.4                             # паддинг вокруг bbox перед ArcFace
ARC_SIZE = 112                              # размер для ArcFace (112x112)
THRESHOLD = 60.0                            # порог "это целевой" в %
FRAME_SKIP = 1                              # 1 = каждый кадр; 2 = каждый 2-й (быстрее)

BLUR_KERNEL = (35, 35)                      # сила блюра (должны быть НЕЧЁТНЫЕ числа)
BLUR_EXPAND = 0.10                          # расширить bbox лица для блюра (10%)

# =========================
# 2) УСТРОЙСТВО ДЛЯ ArcFace
# =========================
device = 0 if torch.cuda.is_available() else "cpu"  # GPU для ArcFace, если доступен
print("🔥 ArcFace использует устройство:", device)

# =========================
# 3) ЗАГРУЗКА МОДЕЛЕЙ
# =========================
yolo = YOLO(YOLO_WEIGHTS)                   # YOLO модель (будем вызывать на CPU)

face_app = FaceAnalysis(                    # InsightFace app (даёт embedding)
    name="buffalo_l",
    providers=["CPUExecutionProvider"] if device == "cpu" else ["CUDAExecutionProvider"]
)
face_app.prepare(
    ctx_id=device if device != "cpu" else -1,  # GPU id или CPU
    det_size=(640, 640)                        # размер окна
)

# =========================
# 4) ЦЕЛЕВОЙ ЭМБЕДДИНГ
# =========================
target_emb = np.load(TARGET_EMB_PATH)       # загрузка эталона
target_emb = target_emb / np.linalg.norm(target_emb)  # L2-нормализация

# =========================
# 5) ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# =========================
def crop_with_padding(img, box, pad_ratio):
    """Обрезаем лицо по bbox + паддинг (для более стабильного embedding)."""
    h, w = img.shape[:2]                    # размеры кадра
    x1, y1, x2, y2 = map(int, box)          # bbox -> int
    bw, bh = x2 - x1, y2 - y1               # ширина/высота bbox
    pw, ph = int(bw * pad_ratio), int(bh * pad_ratio)  # паддинг
    x1 = max(0, x1 - pw)                    # clamp к границам кадра
    y1 = max(0, y1 - ph)
    x2 = min(w, x2 + pw)
    y2 = min(h, y2 + ph)
    return img[y1:y2, x1:x2]                # возвращаем кроп

def make_square(img):
    """Делаем квадратный кроп, не искажая пропорции (добавляем чёрные поля)."""
    h, w = img.shape[:2]                    # размеры кропа
    size = max(h, w)                        # сторона квадрата
    square = np.zeros((size, size, 3), dtype=np.uint8)  # чёрный холст
    y = (size - h) // 2                     # вертикальный сдвиг для центрирования
    x = (size - w) // 2                     # горизонтальный сдвиг для центрирования
    square[y:y+h, x:x+w] = img              # вставляем исходный кроп в центр
    return square

def cosine_to_percent(cos_sim):
    """Cosine similarity (-1..1) -> процент (0..100)."""
    return (cos_sim + 1) / 2 * 100

def expand_box(box, img_w, img_h, expand_ratio=0.10):
    """Расширяем bbox для блюра, чтобы закрывать края лица/волосы."""
    x1, y1, x2, y2 = map(int, box)
    bw, bh = x2 - x1, y2 - y1
    ex = int(bw * expand_ratio)
    ey = int(bh * expand_ratio)
    x1 = max(0, x1 - ex)
    y1 = max(0, y1 - ey)
    x2 = min(img_w, x2 + ex)
    y2 = min(img_h, y2 + ey)
    return x1, y1, x2, y2

def blur_region(frame, x1, y1, x2, y2, ksize=(35, 35)):
    """Блюрим прямоугольную область кадра."""
    roi = frame[y1:y2, x1:x2]               # берём регион лица
    if roi.size == 0:                       # защита от пустых регионов
        return
    roi_blur = cv2.GaussianBlur(roi, ksize, 0)  # гауссов блюр
    frame[y1:y2, x1:x2] = roi_blur          # возвращаем на место

# =========================
# 6) ОТКРЫВАЕМ ВИДЕО
# =========================
cap = cv2.VideoCapture(VIDEO_PATH)          # читаем видео
fps = cap.get(cv2.CAP_PROP_FPS)             # fps
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # ширина
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # высота

writer = cv2.VideoWriter(                   # writer для сохранения результата
    OUT_VIDEO,
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (W, H)
)

print("▶️ Обработка видео (blur всех кроме целевого)...")

# =========================
# STATS (счётчики)
# =========================
total_faces_found = 0        # сколько всего лиц нашёл YOLO (по всем кадрам)
total_faces_blurred = 0      # сколько лиц заблюрили (по всем кадрам)
total_faces_kept = 0         # сколько лиц оставили (обычно 0 или 1 на кадр)

frames_with_faces = 0        # сколько кадров, где YOLO нашёл хотя бы 1 лицо
frames_with_target = 0       # сколько кадров, где нашли целевое лицо (best_score >= THRESHOLD)
frames_processed = 0         # сколько кадров реально обработано (с учётом FRAME_SKIP)

# =========================
# 7) ОСНОВНОЙ ЦИКЛ ПО КАДРАМ
# =========================
frame_id = 0                                # счётчик кадров
while True:
    ret, frame = cap.read()                 # читаем кадр
    if not ret:                             # если конец видео
        break

    frame_id += 1                           # увеличиваем счётчик

    # Пропуск кадров (ускорение)
    if FRAME_SKIP > 1 and (frame_id % FRAME_SKIP != 0):
        writer.write(frame)                 # записываем без обработки
        continue

    # Учитываем обработанный кадр (после FRAME_SKIP)
    frames_processed += 1

    # =========================
    # YOLO (CPU) детекция лиц
    # =========================
    results = yolo(frame, conf=0.25, device="cpu", verbose=False)[0]  # детект
    if results.boxes is None:               # если нет боксов
        writer.write(frame)                 # сохраняем кадр как есть
        continue

    boxes = results.boxes.xyxy.cpu().numpy()  # bbox'ы лиц (N,4)

    # =========================
    # STATS: лица на кадре
    # =========================
    if len(boxes) > 0:
        frames_with_faces += 1
    total_faces_found += len(boxes)

    # =========================
    # 7.1) Считаем похожесть каждого лица с целевым
    # =========================
    similarities = []                       # список похожестей для каждого box

    for box in boxes:                       # для каждого лица
        face = crop_with_padding(frame, box, PAD_RATIO)  # кроп с паддингом
        if face.size == 0:                  # если кроп пустой
            similarities.append(-1.0)       # отметим как невалидный
            continue

        face_sq = make_square(face)         # делаем квадрат
        face_sq = cv2.resize(face_sq, (ARC_SIZE, ARC_SIZE))  # resize
        face_rgb = cv2.cvtColor(face_sq, cv2.COLOR_BGR2RGB)  # BGR->RGB

        faces = face_app.get(face_rgb)      # получаем embedding
        if len(faces) == 0:                 # если ArcFace не смог
            similarities.append(-1.0)
            continue

        emb = faces[0].embedding            # берём embedding
        emb = emb / np.linalg.norm(emb)     # нормируем

        cos_sim = float(np.dot(target_emb, emb))  # cosine similarity
        sim_percent = cosine_to_percent(cos_sim)  # в проценты
        similarities.append(sim_percent)    # сохраняем

    # =========================
    # 7.2) Выбираем "целевое лицо" на кадре (самое похожее)
    # =========================
    best_idx = int(np.argmax(similarities)) if len(similarities) else -1  # индекс лучшего
    best_score = similarities[best_idx] if best_idx >= 0 else -1.0        # его score

    # Если на кадре есть уверенное совпадение — оставим его НЕ заблюренным
    keep_idx = best_idx if best_score >= THRESHOLD else -1                # иначе блюрим всех

    # =========================
    # STATS: найден ли target на кадре
    # =========================
    if keep_idx != -1:
        frames_with_target += 1

    # =========================
    # 7.3) Блюрим всех, кроме keep_idx
    # =========================
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = expand_box(box, W, H, BLUR_EXPAND)  # расширяем область

        if i != keep_idx:                                    # если НЕ целевой
            blur_region(frame, x1, y1, x2, y2, BLUR_KERNEL)  # блюрим лицо
            total_faces_blurred += 1                          # STATS: заблюрили
        else:
            total_faces_kept += 1                             # STATS: оставили (не блюрим)

            # (не обязательно) подпишем целевого
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                frame,
                f"TARGET {best_score:.1f}%",
                (x1, max(20, y1 - 10)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2
            )

    # записываем кадр в выходное видео
    writer.write(frame)

# =========================
# 8) ЗАКРЫВАЕМ ФАЙЛЫ
# =========================
cap.release()                               # закрываем видео
writer.release()                            # закрываем writer
print("✅ Готово! Видео сохранено:", OUT_VIDEO)

# =========================
# 9) ПЕЧАТЬ СТАТИСТИКИ
# =========================
print("\n📊 STATS:")
print(f"Frames read total:           {frame_id}")
print(f"Frames processed (skip):     {frames_processed}")
print(f"Frames with faces:           {frames_with_faces}")
print(f"Frames with target kept:     {frames_with_target}")

print(f"\nTotal faces found (YOLO):    {total_faces_found}")
print(f"Total faces blurred:         {total_faces_blurred}")
print(f"Total faces NOT blurred:     {total_faces_kept}")

if total_faces_found > 0:
    print(f"\nKept rate (not blurred):     {100.0 * total_faces_kept / total_faces_found:.2f}%")
    print(f"Blur rate:                   {100.0 * total_faces_blurred / total_faces_found:.2f}%")


🔥 ArcFace использует устройство: 0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvide

In [2]:
# =====================================================
# BLUR ВСЕХ ЛИЦ КРОМЕ ЦЕЛЕВОГО (без YOLO)
# InsightFace: детекция + эмбеддинги (ArcFace)
# =====================================================

import os                              # работа с файлами/папками
import cv2                             # видео/картинки
import numpy as np                     # математика/массивы
import urllib.request                  # скачать target по URL
import subprocess                      # ffprobe/ffmpeg
import yt_dlp                          # скачать видео с YouTube
from insightface.app import FaceAnalysis  # InsightFace пайплайн

# ===============================
# 0) НАСТРОЙКИ
# ===============================

# URL/путь к целевому лицу (можно заменить на локальный файл)
TARGET_URL = "https://ichef.bbci.co.uk/ace/standard/976/cpsprodpb/b7b3/live/66207fb0-bfbf-11f0-8db2-d92fbd46f955.jpg.webp"
TARGET_IMG = "target_face.jpg"

# YouTube видео
VIDEO_URL = "https://www.youtube.com/watch?v=TV0lRfxdsRg"
INPUT_VIDEO = "input_video.webm"      # yt-dlp может сохранить webm
OUTPUT_VIDEO = "blur_except_target.mp4"

# Порог похожести (cosine). Чем выше — тем строже “это точно целевой”
SIM_THRESHOLD = 0.45

# Минимальный размер лица (чтобы не блюрить мусор/микро-лица)
MIN_FACE_SIZE = 50  # px

# Сила блюра (чем больше kernel, тем сильнее)
BLUR_KERNEL = (35, 35)

# Овал чуть меньше/больше бокса (множитель полуосей)
ELLIPSE_SCALE = 1.05  # 1.0 = ровно бокс, 1.1 = немного шире

# Частота логов
LOG_EVERY = 15  # лог каждые N кадров

# ===============================
# 1) СКАЧАТЬ ЦЕЛЕВОЕ ЛИЦО
# ===============================

if not os.path.exists(TARGET_IMG):
    print("📥 Downloading target face...")
    urllib.request.urlretrieve(TARGET_URL, TARGET_IMG)
else:
    print("✓ Target face already exists")

# ===============================
# 2) СКАЧАТЬ ВИДЕО
# ===============================

def download_youtube(url: str, filename: str) -> str:
    """Скачивает видео в файл (часто будет .webm)."""
    if os.path.exists(filename):
        return filename

    print("📥 Downloading video...")
    opts = {
        "format": "bestvideo[height<=720]+bestaudio/best",
        "outtmpl": filename,
        "noplaylist": True
    }
    with yt_dlp.YoutubeDL(opts) as ydl:
        ydl.download([url])

    return filename

INPUT_VIDEO = download_youtube(VIDEO_URL, INPUT_VIDEO)

# ===============================
# 3) WEBM/AV1 -> MP4(H264) (чтобы OpenCV и запись были стабильны)
# ===============================

def to_h264_mp4(path: str) -> str:
    """
    Делает MP4(H.264 + AAC) из любого входа (webm/mp4/av1 и т.п.).
    Возвращает путь к mp4.
    """
    out = os.path.splitext(path)[0] + "_h264.mp4"

    # Если уже есть — используем
    if os.path.exists(out):
        return out

    print("🎞 Converting to H.264 MP4 for stable processing...")

    subprocess.run([
        "ffmpeg", "-y",
        "-i", path,
        "-c:v", "libx264",
        "-preset", "fast",
        "-crf", "23",
        "-pix_fmt", "yuv420p",
        "-c:a", "aac",
        "-movflags", "+faststart",
        out
    ], check=True)

    return out

SAFE_INPUT = to_h264_mp4(INPUT_VIDEO)

# ===============================
# 4) ИНИЦИАЛИЗАЦИЯ INSIGHTFACE
# ===============================

print("🧠 Loading InsightFace (FaceAnalysis)...")

# name="buffalo_l" — популярный набор (детектор + распознавание)
face_app = FaceAnalysis(
    name="buffalo_l",
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"]  # GPU если есть
)

# det_size влияет на качество/скорость детекции (больше = лучше, но медленнее)
face_app.prepare(ctx_id=0, det_size=(640, 640))

# ===============================
# 5) УТИЛИТЫ: эмбеддинг и cosine similarity
# ===============================

def l2norm(v: np.ndarray) -> np.ndarray:
    """Нормализация вектора."""
    n = np.linalg.norm(v)
    return v / (n + 1e-12)

def cosine_sim(a: np.ndarray, b: np.ndarray) -> float:
    """Косинусная похожесть для L2-нормированных векторов."""
    return float(np.dot(a, b))

def get_target_embedding(bgr_img: np.ndarray) -> np.ndarray:
    """
    Находит лицо на целевом фото, берёт эмбеддинг (первое/самое уверенное).
    """
    rgb = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
    faces = face_app.get(rgb)

    if len(faces) == 0:
        raise RuntimeError("❌ Не удалось найти лицо на TARGET_IMG")

    # Берём самое большое лицо (на всякий случай)
    faces = sorted(
        faces,
        key=lambda f: (f.bbox[2] - f.bbox[0]) * (f.bbox[3] - f.bbox[1]),
        reverse=True
    )

    emb = faces[0].embedding
    return l2norm(emb)

# ===============================
# 6) ЭМБЕДДИНГ ЦЕЛЕВОГО ЛИЦА
# ===============================

target_bgr = cv2.imread(TARGET_IMG)
if target_bgr is None:
    raise RuntimeError("❌ Не удалось прочитать TARGET_IMG (путь/формат)")

target_emb = get_target_embedding(target_bgr)
print("✓ Target embedding extracted")

# ===============================
# 7) ОВАЛЬНЫЙ BLUR ВНУТРИ BBOX
# ===============================

def oval_blur_roi(frame_bgr: np.ndarray, x1: int, y1: int, x2: int, y2: int) -> None:
    """
    Размывает область (x1,y1,x2,y2) на frame_bgr с овальной маской.
    Меняет frame_bgr "на месте".
    """
    # Вырезаем ROI (лицо)
    roi = frame_bgr[y1:y2, x1:x2]
    if roi.size == 0:
        return

    # Делаем размытую версию ROI
    blurred = cv2.GaussianBlur(roi, BLUR_KERNEL, 0)

    # Создаём маску того же размера (3 канала)
    mask = np.zeros_like(roi, dtype=np.uint8)

    rh, rw = roi.shape[:2]
    center = (rw // 2, rh // 2)

    # полуоси овала (можно чуть расширить)
    ax = max(1, int((rw / 2) * ELLIPSE_SCALE))
    ay = max(1, int((rh / 2) * ELLIPSE_SCALE))

    # Рисуем залитый овал
    cv2.ellipse(
        mask,
        center=center,
        axes=(ax, ay),
        angle=0,
        startAngle=0,
        endAngle=360,
        color=(255, 255, 255),
        thickness=-1
    )

    # Превращаем маску в float [0..1]
    m = mask.astype(np.float32) / 255.0

    # Смешиваем: внутри овала — blurred, снаружи — original
    mixed = (blurred * m + roi * (1.0 - m)).astype(np.uint8)

    # Возвращаем в кадр
    frame_bgr[y1:y2, x1:x2] = mixed

# ===============================
# 8) ОБРАБОТКА ВИДЕО
# ===============================

cap = cv2.VideoCapture(SAFE_INPUT)
if not cap.isOpened():
    raise RuntimeError("❌ Не удалось открыть видео")

fps = cap.get(cv2.CAP_PROP_FPS)
fw  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fh  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Пишем mp4
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (fw, fh))

frame_id = 0
kept = 0
blurred_cnt = 0

print("🎬 Processing video (blur all except target)...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1

    # InsightFace ожидает RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Находим все лица на кадре
    faces = face_app.get(rgb)

    # Для каждого лица решаем: блюрить или оставить
    for f in faces:
        x1, y1, x2, y2 = map(int, f.bbox)

        # Подрежем координаты по кадру
        x1 = max(0, x1); y1 = max(0, y1)
        x2 = min(frame.shape[1], x2); y2 = min(frame.shape[0], y2)

        w = x2 - x1
        h = y2 - y1

        # Фильтр по размеру
        if w < MIN_FACE_SIZE or h < MIN_FACE_SIZE:
            continue

        # Эмбеддинг этого лица (InsightFace уже даёт embedding для каждого face)
        emb = l2norm(f.embedding)

        # Сравнение с target
        sim = cosine_sim(target_emb, emb)

        # Если это НЕ целевой — блюрим
        if sim < SIM_THRESHOLD:
            oval_blur_roi(frame, x1, y1, x2, y2)
            blurred_cnt += 1
        else:
            kept += 1

    # Записываем кадр в выходное видео
    writer.write(frame)

    if frame_id % LOG_EVERY == 0:
        print(f"➡ frame {frame_id}: kept={kept}, blurred={blurred_cnt}, faces={len(faces)}")

cap.release()
writer.release()

print("🎉 DONE")
print("📹 Video saved to:", OUTPUT_VIDEO)
print(f"✅ kept (target-like): {kept}")
print(f"✅ blurred (others):   {blurred_cnt}")


✓ Target face already exists
🧠 Loading InsightFace (FaceAnalysis)...
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied

### Шаг 7. Упаковка в Docker
- Создание Dockerfile с необходимыми библиотеками и зависимостями  
- Развёртывание проекта как API или самостоятельного сервиса  
- Тестирование контейнера на разных платформах и видеоматериалах

In [ ]:
# Use official Python base image with CUDA support for GPU acceleration
FROM nvidia/cuda:11.8.0-cudnn8-runtime-ubuntu22.04

# Set working directory
WORKDIR /app

# Prevent interactive prompts during installation
ENV DEBIAN_FRONTEND=noninteractive

# Install system dependencies
RUN apt-get update && apt-get install -y \
    python3.10 \
    python3-pip \
    libglib2.0-0 \
    libsm6 \
    libxext6 \
    libxrender-dev \
    libgomp1 \
    libgl1-mesa-glx \
    wget \
    && rm -rf /var/lib/apt/lists/*

# Upgrade pip
RUN pip3 install --upgrade pip

# Install Python dependencies
RUN pip3 install --no-cache-dir \
    ultralytics \
    opencv-python-headless \
    opencv-python \
    matplotlib \
    scipy \
    numpy \
    torch \
    torchvision \
    pyyaml

# Copy project files
COPY . /app

# Create directories for input/output
RUN mkdir -p /app/input /app/output /app/models

# Download pre-trained model (if needed)
# RUN wget -O /app/models/best.pt <your-model-url>

# Set environment variables
ENV PYTHONUNBUFFERED=1
ENV MODEL_PATH=/app/models/best.pt

# Expose port for API (if needed)
EXPOSE 8000

# Default command - can be overridden
CMD ["python3", "app.py"]

In [ ]:
import os

os.makedirs('/root/faceblur', exist_ok=True)
os.chdir('/root/faceblur')
print(f"✓ Current directory: {os.getcwd()}")

In [ ]:
# Create Dockerfile
dockerfile_content = """# Use official Python base image with CUDA support for GPU acceleration
FROM nvidia/cuda:11.8.0-cudnn8-runtime-ubuntu22.04

# Set working directory
WORKDIR /app

# Prevent interactive prompts during installation
ENV DEBIAN_FRONTEND=noninteractive

# Install system dependencies
RUN apt-get update && apt-get install -y \\
	python3.10 \\
	python3-pip \\
	libglib2.0-0 \\
	libsm6 \\
	libxext6 \\
	libxrender-dev \\
	libgomp1 \\
	libgl1-mesa-glx \\
	wget \\
	&& rm -rf /var/lib/apt/lists/*

# Upgrade pip
RUN pip3 install --upgrade pip

# Install Python dependencies
RUN pip3 install --no-cache-dir \\
	ultralytics \\
	opencv-python-headless \\
	opencv-python \\
	matplotlib \\
	scipy \\
	numpy \\
	torch \\
	torchvision \\
	pyyaml

# Copy project files
COPY . /app

# Create directories for input/output
RUN mkdir -p /app/input /app/output /app/models

# Download pre-trained model (if needed)
# RUN wget -O /app/models/best.pt <your-model-url>

# Set environment variables
ENV PYTHONUNBUFFERED=1
ENV MODEL_PATH=/app/models/best.pt

# Expose port for API (if needed)
EXPOSE 8000

# Default command - can be overridden
CMD ["python3", "app.py"]
"""

with open('Dockerfile', 'w') as f:
	f.write(dockerfile_content)

print("✓ Dockerfile created successfully!")


In [ ]:
# Create app.py with face detection and blur functionality
app_content = """#!/usr/bin/env python3
import os
import cv2
import numpy as np
from ultralytics import YOLO

# Load model
MODEL_PATH = os.getenv('MODEL_PATH', '/app/models/best.pt')
model = YOLO(MODEL_PATH)

def blur_rounded(img, box, blur_k=61, radius=40):
	x1, y1, x2, y2 = map(int, box)
	face = img[y1:y2, x1:x2].copy()
	
	blurred = cv2.GaussianBlur(face, (blur_k, blur_k), 0)
	
	h, w = face.shape[:2]
	mask = np.zeros((h, w), dtype=np.uint8)
	
	cv2.rectangle(mask, (radius, 0), (w-radius, h), 255, -1)
	cv2.rectangle(mask, (0, radius), (w, h-radius), 255, -1)
	cv2.circle(mask, (radius, radius), radius, 255, -1)
	cv2.circle(mask, (w-radius, radius), radius, 255, -1)
	cv2.circle(mask, (radius, h-radius), radius, 255, -1)
	cv2.circle(mask, (w-radius, h-radius), radius, 255, -1)
	
	mask3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR) / 255.0
	blended = (blurred * mask3 + face * (1 - mask3)).astype(np.uint8)
	
	img[y1:y2, x1:x2] = blended
	return img

def process_image(input_path, output_path):
	img = cv2.imread(input_path)
	if img is None:
		print(f"Error: Could not read {input_path}")
		return
	
	results = model(img)[0]
	
	for box in results.boxes.xyxy:
		img = blur_rounded(img, box)
	
	cv2.imwrite(output_path, img)
	print(f"Processed: {output_path}")

if __name__ == "__main__":
	input_dir = "/app/input"
	output_dir = "/app/output"
	
	os.makedirs(output_dir, exist_ok=True)
	
	for filename in os.listdir(input_dir):
		if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
			input_path = os.path.join(input_dir, filename)
			output_path = os.path.join(output_dir, filename)
			process_image(input_path, output_path)
	
	print("✓ All images processed!")
"""

with open('app.py', 'w') as f:
	f.write(app_content)

print("✓ app.py created successfully!")


In [ ]:
from ultralytics import YOLO
import os

print("🔥 Model running inside Docker!")
model = YOLO("/root/runs/detect/train5/weights/best.pt")

# Use an actual image from the validation dataset
test_image = os.path.join(img_val, os.listdir(img_val)[0])
print(f"Testing with image: {test_image}")

results = model(test_image)
print(results)
print("✓ Model inference completed!")

In [ ]:
%pip install fastapi uvicorn

In [ ]:
%pip install python-multipart

from fastapi import FastAPI, UploadFile
from ultralytics import YOLO
import cv2
import numpy as np

app = FastAPI()
model = YOLO("/root/runs/detect/train5/weights/best.pt")

@app.post("/blur_face/")
async def blur_face(file: UploadFile):
    data = await file.read()
    npimg = np.frombuffer(data, np.uint8)
    img = cv2.imdecode(npimg, cv2.IMREAD_COLOR)

    results = model(img)[0]

    for box in results.boxes.xyxy:
        x1, y1, x2, y2 = map(int, box)
        img = cv2.GaussianBlur(img, (61,61), 0)

    _, img_encoded = cv2.imencode(".jpg", img)
    return {"image": img_encoded.tobytes().hex()}
